# Script to Process Upper Columbia Limiting Factor Data Based on Criteria 
# Data from RTT Prioritizatoin Process - August 28th
## Author: Ryan Niemeyer, Upper Columbia Salmon Recovery Board
### For more information, see https://www.ucsrb.org/prioritization/


In [244]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://storage.googleapis.com/ff-storage-p01/festivals/logos/000/051/750/large/logo.jpg?1575572027", width=200, height=200)

In [1]:
## -------------------- NEXT ACTION ----------------------

# 3) Develop output table that has the percentage per reach (see screenshot I took - https://docs.google.com/document/d/15zJqUM0gwMeFxR5rf9Pli40A45CYo9cNpAjKQa1oXTA/edit)
#     3A) make sure all steps are completed (see 8 steps below)
#     3B) chcek that the "Step2_Limiting_Factors_Score_Spring_Chinook" and Steelhead is the correct output (same as the table Greer shared)
#     3C) color code the tables (red-yellow-green AND color based on life stage) and save as "Master" or "checkpoint" to match what Greer shared

## ------------------------------------ Upload Python Libraries -----------------------------------

In [1]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt 
pd.options.display.max_colwidth = 150 # make it so print statements can be longer
pd.options.display.max_columns = None
pd.options.display.max_rows = 200
# CHEAT SHEET for pd and np commands: https://www.dataquest.io/blog/pandas-cheat-sheet/#:~:text=%20Pandas%20Cheat%20Sheet%20%E2%80%94%20Python%20for%20Data,CSV%2C%20.xlsx%2C%20SQL%2C%20or%20JSON.%20%20More%20
# to print entire data frame: pd.set_option('display.max_rows', None)

## ---------------------------------------------------------------------------------------------------------------------------

# -
# Choose the Criteria for Limiting Factor Output Table and Output
# -

## ---------------------------------------------------------------------------------------------------------------------------
### !!!!! DOUBLE CHECK these are the criteria you want !!!!!!!!!!!!!!

## ---------------------------------------------------------------------------------------------------------------------------
## Criteria for Query: 
### Update if criteria for output habitat quality tables change
## ---------------------------------------------------------------------------------------------------------------------------

In [537]:
# -------------------------------------------------------------------------------------------------------------
#
#        Variables that are for ANY life stage
#
# -------------------------------------------------------------------------------------------------------------

# -------- Assessment Unit (AU) (HUC12) Tier rank (data in habitat_attribute_scores) ----------------
AU_Rank = [1]                        # 1 is high priority, 3 is low priority

# --------------------- Habitat Attribute Scores-- "DRAFT Habitat Attribute..." data (data in Cumul_Habitat_Attribute_Score)
# Oct 2020: did [1] (Acceptable only)   OR [1, 3] (Acceptable and At Risk)
Cumul_Habitat_Attribute_Score = [1]   # possible scores - [1,3,5] 1 - unacceptable, 5 - adequate

# --------------- Sum Life Stage (sum of all the life stages present in reach) ---------------------
Sum_Life_Stage = [4, 5, 6, 7]   # number of life stages present in a reach

# -------------------------------------------------------------------------------------------------------------
#
#        Variables for SPECIFIC life stages
#
# -------------------------------------------------------------------------------------------------------------

# -------------------- Life Stage Priority (LSP) rank is "High Priority" (data in AU_Life_Stage_Priorities)  -------------------
Life_Stage_Priority = ["High Priority"]
# for test: 
#Life_Stage_Priority =  ["Life Stage Not Supported", "Low Priority", "Medium Priority", "High Priority"] 

# -------------------- Life Stage is Present is 1 (data in reach_life_stages)  -------------------
Reach_Life_Stage_Presence = [1]
# for tests: 
# Reach_Life_Stage_Presence =  [0, 1]

## ---------------------------------------------------------------------------------------------------------------------------
## Tables and Column Names of Indicators/Variables: 
### Update if excel input names change OR column names change
## ---------------------------------------------------------------------------------------------------------------------------

In [538]:
# -------------------------------------------------------------------------------------------------------------
#
#        Variables that are for ANY life stage
#
# -------------------------------------------------------------------------------------------------------------

# ------ AU_rank_vars = habitat_attribute_scores = 'AU Tier Spring CHN' and 'AU Tier Steelhead'
AU_Rank_vars = { 
 'data_frame': ['habitat_attribute_scores', 'habitat_attribute_scores'  ], 
 'variable': ['AU Tier Spring CHN' , 'AU Tier Steelhead'] 
} 
# create a dataframe 
AU_Rank_vars = pd.DataFrame(AU_Rank_vars, columns = ['data_frame', 'variable']) 

# ------ Habitat_Attribute_score: habitat_attribute_scores = 'DRAFT Habitat Attribute Score (Currently MAX Score OR "0" if NA)'
Cumul_Habitat_Attribute_Score_vars = { 
 'data_frame': ['Cumul_Habitat_Attribute_Score' ], 
 'variable': ['DRAFT Habitat Attribute Score (Currently MIN Score OR NA)'] 
} 
# create a dataframe 
Cumul_Habitat_Attribute_Score_vars = pd.DataFrame(Cumul_Habitat_Attribute_Score_vars, columns = ['data_frame', 'variable']) 


# -------------------------------------------------------------------------------------------------------------
#
#       Variables for SPECIFIC life stages
#
# -------------------------------------------------------------------------------------------------------------

# ------ AU Life Stage Priority: AU_Life_Stage_Priorities = the life stages-------
Life_Stage_Priority_vars = { 
 'data_frame': ['AU_Life_Stage_Priorities','AU_Life_Stage_Priorities','AU_Life_Stage_Priorities','AU_Life_Stage_Priorities',
               'AU_Life_Stage_Priorities','AU_Life_Stage_Priorities','AU_Life_Stage_Priorities'], 
 'variable_SPCH': ['SPCH_AdultMigrationLSP', 'SPCH_HoldingLSP', 'SPCH_SpawnLSP', 'SPCH_FryLSP', 'SPCH_SummerRearingLSP',
              'SPCH_WinterRearingLSP', 'SPCH_SmoltLSP'],
     'variable_SH': ['SH_AdultMigrationLSP','SH_HoldingLSP','SH_SpawningLSP','SH_FryLSP','SH_SummerRearingLSP',
                       'SH_WinterRearingLSP','SH_SmoltLSP'],
     'variable': ['Adult Migration','Holding and Maturation','Spawning','Fry','Summer Rearing',
                   'Winter Rearing','Smolt']
} 
# create a dataframe 
Life_Stage_Priority_vars = pd.DataFrame(Life_Stage_Priority_vars, columns = ['data_frame', 'variable_SPCH','variable_SH','variable']) 


# ------ Reach Life Stage: reach_life_stages = the life stages -------
Reach_Life_Stage_Presence_vars = { 
 'data_frame': ['Reach_Life_Stage_Presence','Reach_Life_Stage_Presence','Reach_Life_Stage_Presence','Reach_Life_Stage_Presence',
               'Reach_Life_Stage_Presence','Reach_Life_Stage_Presence','Reach_Life_Stage_Presence'], 
 'variable_SPCH': ['SPCH_Adult_Migration','SPCH_Holding','SPCH_Spawning','SPCH_Fry_Colonization','SPCH_Summer_Rearing',
                   'SPCH_Winter_Rearing','SPCH_Smolt_Emigration'],
    
     'variable_SH': ['SH_Adult_Migration', 'SH_Holding', 'SH_Spawning','SH_Fry_Colonization', 'SH_Summer_Rearing', 
                     'SH_Winter_Rearing', 'SH_Smolt_Emigration'] 
} 
# create a dataframe 
Reach_Life_Stage_Presence_vars = pd.DataFrame(Reach_Life_Stage_Presence_vars, columns = ['data_frame', 'variable_SPCH','variable_SH']) 

# ------ Life Stage Habitat Attributes: life_stage_habitat = Life Stage -------
Life_Stage_Habitat_Attributes_vars = { 
 'data_frame': ['life_stage_habitat','life_stage_habitat','life_stage_habitat','life_stage_habitat',
               'life_stage_habitat','life_stage_habitat','life_stage_habitat'], 
 'variable': ['Adult Migration', 'Holding and Maturation', 'Spawning and Incubation', 'Fry',
                    'Summer Rearing', 'Winter Rearing', 'Smolt Outmigration'] 
} 
# create a dataframe 
Life_Stage_Habitat_Attributes_vars = pd.DataFrame(Life_Stage_Habitat_Attributes_vars, columns = ['data_frame', 'variable']) 



# -------------------- List the life stages --------------
life_stages_all = {                   'life_stages': ['Adult Migration','Holding and Maturation','Spawning','Fry','Summer Rearing',
                   'Winter Rearing','Smolt']
} 
# create data frame 
life_stages_all = pd.DataFrame(life_stages_all, columns = ['life_stages']) 


# ------ Sum_Life_Stage:   reach_life_stages = 'Sum_LifeStage'
Sum_Life_Stage_vars = { 
 'data_frame': ['reach_life_stages'  ], 
 'variable': ['TotalNum_LS']
    
} 
# create a dataframe 
Sum_Life_Stage_vars = pd.DataFrame(Sum_Life_Stage_vars, columns = ['data_frame', 'variable']) 




## ---------------------------------------------------------------------------------------------------------------------------
## Output Locations:
### Update if you want output path to change
## ---------------------------------------------------------------------------------------------------------------------------

In [539]:
# ---------- output directory ------------------------------
# NOTE: just need to change this directory, remaining script will generate pathways automatically
output_location = 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/'

# ------------------------ Spring Chinook output paths ---------------
species_x = "Spring_Chinook"
Output_Spring_Chinook_list = []
for i in range(life_stages_all.shape[0]):
    
    # ------- generate life stage name for file name -----
    life_stage_output_string = str(life_stages_all.loc[i][0]).replace(" ", "_")
    # --------- generate path for life stage output ----------
    pathx = output_location + "Limiting_Factor_Output," + species_x + ","  + life_stage_output_string + ".xlsx"
    # for all reaches pathx = output_location + "Limiting_Factor_Output," + species_x + ",ALL_REACHES,"  + life_stage_output_string + ".xlsx"
    # -------- append directory path to list ------
    Output_Spring_Chinook_list.append(pathx)
    

# ------------------------ Steelhead output paths ---------------
species_x = "Steelhead"
Output_Steelhead_list = []
for i in range(life_stages_all.shape[0]):
    
    # ------- generate life stage name for file name -----
    life_stage_output_string = str(life_stages_all.loc[i][0]).replace(" ", "_")
    # --------- generate path for life stage output ----------
    pathx = output_location + "Limiting_Factor_Output," + species_x + ","  + life_stage_output_string + ".xlsx"
    # for all reachespathx = output_location + "Limiting_Factor_Output," + species_x + ",ALL_REACHES,"  + life_stage_output_string + ".xlsx"
    # -------- append directory path to list ------
    Output_Steelhead_list.append(pathx)
    
    

print('output directory: ' )
print(Output_Steelhead_list)

# ------------------------ Output the combined limting factor (for each species) ----------
output_location_master_spring_chinook = output_location + 'Limiting_Factor,Spring_Chinook,all_Life_Stages,ALL_SCORES.xlsx'
output_location_master_steelhead = output_location + 'Limiting_Factor,Steelhead,all_Life_Stages,ALL_SCORES.xlsx'

# ---------------------------- Output Life Stage Habitat Quality Assessment for both species -------------
output_location_life_stage_assessment_spring_chinook = output_location + 'Life_Stage_Habitat_Quality_Assessment_Spring_Chinook.xlsx'
output_location_life_stage_assessment_steelhead = output_location + 'Life_Stage_Habitat_Quality_Assessment_Steelhead.xlsx'


output directory: 
['Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/Limiting_Factor_Output,Steelhead,Adult_Migration.xlsx', 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/Limiting_Factor_Output,Steelhead,Holding_and_Maturation.xlsx', 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/Limiting_Factor_Output,Steelhead,Spawning.xlsx', 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/Limiting_Factor_Output,Steelhead,Fry.xlsx', 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/Limiting_Factor_Output,Steelhead,Summer_Rearing.xlsx', 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/Limiting_Factor_Output,Steelhead,Winter_Rearing.xlsx', 'Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/Limiting_Factor_Output,Steelhead,Smolt.xlsx']


## ---------------------------------------------------------------------------------------------------------------------------
##  Limiting Factor Percent: Scoring Rules
###  Update to reflect changes in percentage of reaches that have habitat attribute that is a limiting factor
## ---------------------------------------------------------------------------------------------------------------------------

In [435]:
#--------------- The three scores possible ------------
scores_limiting_factors_prcnt_reaches = [1, 3, 5]

# --------------- ranges for percent of reaches in an Assessment Unit to quality for three scores ------------
score_cutoff_percent_of_reaches = { 
 'cut_off': [0.3,0.6],
} 
# create a dataframe 
score_cutoff_percent_of_reaches = pd.DataFrame(score_cutoff_percent_of_reaches, columns = ['cut_off']) 


## ---------------------------------------------------------------------------------------------------------------------------

# -
# Upload the Habitat Data Excel 
# -

## ---------------------------------------------------------------------------------------------------------------------------

### Open Habitat Analaysis Tabs: Habitat Attribute Score, Confinement Scores

In [436]:
habitat_attribute_scores = pd.read_excel('Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/MASTER_Step2_HabitatAnalysis.xlsx', 
                   'HabitatAttribute_Scores')

In [437]:
habitat_quality_scores = pd.read_excel('Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/MASTER_Step2_HabitatAnalysis.xlsx', 
                   'HabitatQuality_Scores', skiprows=[0])

In [438]:
habitat_attribute_lifestage = pd.read_excel('Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/MASTER_Step2_HabitatAnalysis.xlsx', 
                   'Attribute_LifeStage Crosswalk')

In [439]:
# -------------- strip white space from life stage - habitat attribute data ------------
habitat_attribute_lifestage = habitat_attribute_lifestage.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# ------------------- make it so "Off-Channel Side-Channels" matches exactly with column names ----
# column names have "-" between "Side" and "Channels"
habitat_attribute_lifestage = habitat_attribute_lifestage.replace('Off-Channel- Side Channels', 'Off-Channel- Side-Channels')

In [315]:
# --------- ONLY for testing using data that is on GitHub --------
MASTER_path = 'https://github.com/rniemeyer07/UCSRB_Prioritization_Python_Scripts/blob/master/MASTER_Step2_HabitatAnalysis.xlsx'
MASTER_path = 'https://github.com/rniemeyer07/UCSRB_Prioritization_Python_Scripts/blob/master/HabitatAttribute_Scores.xlsx?raw=true'

habitat_attribute_lifestage_GITHUB = pd.read_excel(MASTER_path, 'HabitatAttribute_Scores')



In [490]:
# habitat_attribute_lifestage[habitat_attribute_lifestage['Life Stage']=="Fry"]

### Open Life Stages Presence by Reach Tab

In [440]:
reach_life_stage_presence_Spring_Chinook = pd.read_excel('Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/MASTER_Step2_HabitatAnalysis.xlsx', 
                   'Reaches Life Stage Spring Chino')
reach_life_stage_presence_Steelhead = pd.read_excel('Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/MASTER_Step2_HabitatAnalysis.xlsx', 
                   'Reaches Life Stage Steelhead')

In [441]:
# ------------ remove any white space at beginning or end of attributes -----------------
reach_life_stage_presence_Spring_Chinook = reach_life_stage_presence_Spring_Chinook.applymap(lambda x: x.strip() if isinstance(x, str) else x)
reach_life_stage_presence_Steelhead = reach_life_stage_presence_Steelhead.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [491]:
# reach_life_stage_presence_Spring_Chinook.loc[reach_life_stage_presence_Spring_Chinook.AssessmentUnit == 'Lower Twisp River']

In [443]:
# habitat_attribute_scores.loc[habitat_attribute_scores.ReachName == 'Entiat River Lake 01']

### Strip white space at beginning and end of Assessment Unit names:

In [540]:
habitat_attribute_scores['Assessment Unit'] = habitat_attribute_scores['Assessment Unit'].str.strip()

### Open Tables from Habitat Database: Reach Life Stage

In [445]:
life_stage_habitat = pd.read_excel('Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/MASTER_Step2_HabitatAnalysis.xlsx', 
                   'Attribute_LifeStage Crosswalk')

In [446]:
AU_life_stage_priorities = pd.read_excel('Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/MASTER_Step2_HabitatAnalysis.xlsx', 
                   'LifeStage_Priorities', skiprows=[0])

### Filter out Habitat Attributes for Life Stages for each Species

In [541]:
Spring_Chinook_Life_Stage_Habitat_Attributes = life_stage_habitat.loc[life_stage_habitat['Species'].isin(['Spring Chinook'])] 
Steelhead_Life_Stage_Habitat_Attributes = life_stage_habitat.loc[life_stage_habitat['Species'].isin(['Steelhead'])] 

In [542]:
# ------------ remove any white space at beginning or end of attributes -----------------
Spring_Chinook_Life_Stage_Habitat_Attributes = Spring_Chinook_Life_Stage_Habitat_Attributes.applymap(lambda x: x.strip() if isinstance(x, str) else x)
Steelhead_Life_Stage_Habitat_Attributes = Steelhead_Life_Stage_Habitat_Attributes.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [543]:
# Steelhead_Life_Stage_Habitat_Attributes['Habitat Attribute'].unique()

In [544]:
# life_stage_habitat['Habitat Attribute'].unique()

## ---------------------------------------------------------------------------------------------------------------------------

# -
# Develop Tables for Individual Life Stages
# -

## ---------------------------------------------------------------------------------------------------------------------------

## ---------------------------------------------------------------------------------------------------------------------------
# Step 1: Filter out for ALL stages (i.e. applies to all life stages)
## ---------------------------------------------------------------------------------------------------------------------------

## ---------------------------------------------------------------------------------------------------------------------------
## Spring Chinook
## ---------------------------------------------------------------------------------------------------------------------------

In [545]:
species_x = 'Spring Chinook'

### Filter out to only have Spring Chinook reaches

In [546]:
spring_chinook_reaches_true = ['yes']
Spring_Chinook_data_frame = habitat_attribute_scores.loc[habitat_attribute_scores['Spring.Chinook.Reach'].isin(spring_chinook_reaches_true)] 
print('initial number of reaches/rows: ' + str(habitat_attribute_scores.shape[0]) )
print('number of reaches after including only Spring Chinook reaches: ' + str(Spring_Chinook_data_frame.shape[0]) )

initial number of reaches/rows: 16888
number of reaches after including only Spring Chinook reaches: 9039


In [547]:
# Spring_Chinook_data_frame.loc[Spring_Chinook_data_frame.ReachName == 'Chiwaukum Creek 01']

### Filter out by AU Rank (Note -  "variable" should be 0 for SprChn and 1 for STLHD)

In [548]:
# NOTE: need to change variable to 0 or 1 (AU_Rank_vars['variable'][1]) based on species 
Spring_Chinook_data_frame = Spring_Chinook_data_frame.loc[Spring_Chinook_data_frame[AU_Rank_vars['variable'][0]].isin(AU_Rank)] 
print('number of reaches/rows after AU criteria: ' + str(Spring_Chinook_data_frame.shape[0]) )

number of reaches/rows after AU criteria: 3105


### Filter out by Sum Life Stage in Reach

In [549]:
Sum_Life_Stage_Criteria = reach_life_stage_presence_Spring_Chinook.loc[reach_life_stage_presence_Spring_Chinook[Sum_Life_Stage_vars['variable'][0]].isin(Sum_Life_Stage)]
Sum_Life_Stage_Criteria_Reaches = Sum_Life_Stage_Criteria.ReachName
Spring_Chinook_data_frame = Spring_Chinook_data_frame.loc[Spring_Chinook_data_frame['ReachName'].isin(Sum_Life_Stage_Criteria_Reaches)] 

# ------------- Prep Sum Life Stage ----------
Sum_Life_Stage_output = Sum_Life_Stage_Criteria.loc[Sum_Life_Stage_Criteria['ReachName'].isin(Spring_Chinook_data_frame.ReachName)] 
Sum_Life_Stage_output = Sum_Life_Stage_output[['ReachName',Sum_Life_Stage_vars['variable'][0]]]
# ------- add life Stage Sum --------------
Spring_Chinook_data_frame = Spring_Chinook_data_frame.sort_values(by = 'ReachName') 
Spring_Chinook_data_frame = pd.merge(Spring_Chinook_data_frame, Sum_Life_Stage_output, on=['ReachName'])

print('number of reaches/row after Sum Life Stage criteria: ' + str(Spring_Chinook_data_frame.shape[0]) )

number of reaches/row after Sum Life Stage criteria: 2783


### Filter out to only include Life Stages with Attributes that meet criteria

In [550]:
Spring_Chinook_data_frame_ALL = Spring_Chinook_data_frame
Spring_Chinook_data_frame = Spring_Chinook_data_frame.loc[Spring_Chinook_data_frame[Cumul_Habitat_Attribute_Score_vars['variable'][0]].isin(Cumul_Habitat_Attribute_Score)] 
print('number of reaches/rows after Habitat Attribute Score criteria:' + str(Spring_Chinook_data_frame.shape[0]) )

number of reaches/rows after Habitat Attribute Score criteria:526


In [551]:
# Spring_Chinook_data_frame[Spring_Chinook_data_frame.ReachName == 'Entiat River Lake 01']#

## ---------------------------------------------------------------------------------------------------------------------------
## Steelhead
## ---------------------------------------------------------------------------------------------------------------------------

In [552]:
species_x = 'Steelhead'

### Filter out to only have Steelhead reaches

In [553]:
steelhead_reaches_true = ['yes']
Steelhead_data_frame = habitat_attribute_scores.loc[habitat_attribute_scores['Steelhead.Reach'].isin(steelhead_reaches_true)] 
print('initial number of reaches/rows: ' + str(habitat_attribute_scores.shape[0]) )
print('number of reaches after including only Steelhead reaches: ' + str(Steelhead_data_frame.shape[0]) )

initial number of reaches/rows: 16888
number of reaches after including only Steelhead reaches: 14000


In [554]:
# Steelhead_data_frame.loc[Steelhead_data_frame.ReachName == 'Roaring Creek Nason 02']

### Filter out by AU Rank (Note -  "variable" should be 0 for SprChn and 1 for STLHD)

In [555]:
# NOTE: need to change variable to 0 or 1 (AU_Rank_vars['variable'][1]) based on species 
Steelhead_data_frame = Steelhead_data_frame.loc[Steelhead_data_frame[AU_Rank_vars['variable'][1]].isin(AU_Rank)] 
print('number of reaches/rows after AU criteria: ' + str(Steelhead_data_frame.shape[0]) )

number of reaches/rows after AU criteria: 5188


### Filter out by Sum Life Stage in Reach

In [556]:
Sum_Life_Stage_Criteria = reach_life_stage_presence_Steelhead.loc[reach_life_stage_presence_Steelhead[Sum_Life_Stage_vars['variable'][0]].isin(Sum_Life_Stage)]
Sum_Life_Stage_Criteria_Reaches = Sum_Life_Stage_Criteria.ReachName
Steelhead_data_frame = Steelhead_data_frame.loc[Steelhead_data_frame['ReachName'].isin(Sum_Life_Stage_Criteria_Reaches)]

# ------------- Prep Sum Life Stage ----------
Sum_Life_Stage_output = Sum_Life_Stage_Criteria.loc[Sum_Life_Stage_Criteria['ReachName'].isin(Steelhead_data_frame.ReachName)] 
Sum_Life_Stage_output = Sum_Life_Stage_output[['ReachName',Sum_Life_Stage_vars['variable'][0]]]
# ------- add life Stage Sum --------------
Steelhead_data_frame = Steelhead_data_frame.sort_values(by = 'ReachName') 
Steelhead_data_frame = pd.merge(Steelhead_data_frame, Sum_Life_Stage_output, on=['ReachName'])
print('number of reaches after Sum Life Stage criteria: ' + str(Steelhead_data_frame.shape[0]) )

number of reaches after Sum Life Stage criteria: 4082


### Filter out to only include Life Stages with Attributes that meet criteria

In [557]:
Steelhead_data_frame = Steelhead_data_frame.loc[Steelhead_data_frame[Cumul_Habitat_Attribute_Score_vars['variable'][0]].isin(Cumul_Habitat_Attribute_Score)] 
print('number of reaches/rows after Habitat Attribute Score criteria:' + str(Steelhead_data_frame.shape[0]) )

number of reaches/rows after Habitat Attribute Score criteria:778


In [558]:
# Steelhead_data_frame.loc[Steelhead_data_frame.ReachName == 'Roaring Creek Nason 02']
# AU_life_stage_priorities[AU_life_stage_priorities.AU == 'Entiat River-Lake Creek']

## ---------------------------------------------------------------------------------------------------------------------------
#  Step 2: Create & Run Function to Output Limiting Factor Tables 
##  Filters out reaches for Specific life stages for each species
## ---------------------------------------------------------------------------------------------------------------------------

## ---------------------------------------------------------------------------------------------------------------------------
##  Create Function
## ---------------------------------------------------------------------------------------------------------------------------

In [466]:
# Steelhead_Life_Stage_Habitat_Attributes['Habitat Attribute'].unique()

In [467]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
 
    return False

# ------ just a dummy array for index values ------------
index_vals = pd.Series(range(0,7))

def Generate_Limiting_Factor_Output_Table(life_stage, species, output_path):

    # ------------------------------------------------------------------------------
    #      Initiate Data Processing
    # ------------------------------------------------------------------------------

    print('------------------- starting output for Species: ' + species + "for the Life Stage: " + life_stage)
    
    # ------------------------------------------------------------------------------
    #       Establish species name variable
    # ------------------------------------------------------------------------------

    if species == "Spring Chinook":

        # ------- establish species variable name -----
        variable_name = 'variable_SPCH'
        # -------------- species data frame --------
        Species_data_frame = Spring_Chinook_data_frame
        # ------------- habitat attribute data frame -----------
        Species_Life_Stage_Habitat_Attributes = Spring_Chinook_Life_Stage_Habitat_Attributes
        # ----------- Use Spring Chinook Reaches-Life Stage data --------
        reach_life_stage_presence = reach_life_stage_presence_Spring_Chinook
        # ---------------- Output correct column names -------------
        output_column_AU_column = 'AU Tier Spring CHN' 

    elif species == "Steelhead":

        # ------- establish species variable name -----
        variable_name = 'variable_SH'
        # -------------- species data frame --------
        Species_data_frame = Steelhead_data_frame
        # ------------- habitat attribute data frame -----------
        Species_Life_Stage_Habitat_Attributes = Steelhead_Life_Stage_Habitat_Attributes
        # ----------- Use Steelhead Reaches-Life Stage data --------
        reach_life_stage_presence = reach_life_stage_presence_Steelhead
        # ---------------- Output correct column names -------------
        output_column_AU_column = 'AU Tier Steelhead' 

    else:

        
        print('Incorrectly entered species name - re-type species name')
        return
        
        
    # --------------------------------- life stage index variable --------------------------------      
    life_stage_val = int(index_vals[life_stages_all.life_stages.isin([life_stage])])
    life_stage_val_list = [life_stage_val]
    
    
    # ------------------------------------------------------------------------------
    #       AU Life Stage Priority for this life stage
    # ------------------------------------------------------------------------------

    AU_Life_Stage_Priority_Criteria = AU_life_stage_priorities.loc[AU_life_stage_priorities[Life_Stage_Priority_vars[variable_name][life_stage_val]].isin(Life_Stage_Priority)]
    AU_Life_Stage_Priority_Criteria_AUs = AU_Life_Stage_Priority_Criteria.AU
    Species_data_frame_Life_Stage = Species_data_frame.loc[Species_data_frame['Assessment Unit'].isin(AU_Life_Stage_Priority_Criteria_AUs)] 
    print('number of reaches/row after AU Life Stage Priority criteria: ' + str(Species_data_frame_Life_Stage.shape[0]) )
    
    # ------------------------------------------------------------------------------
    #       Pull reaches with this Life Stage presence
    # ------------------------------------------------------------------------------

    Reach_Life_Stage_Presence_Criteria = reach_life_stage_presence.loc[reach_life_stage_presence[Reach_Life_Stage_Presence_vars[variable_name][life_stage_val]].isin(Reach_Life_Stage_Presence)]
    Reach_Life_Stage_Presence_Criteria_ReachName = Reach_Life_Stage_Presence_Criteria.ReachName
    Species_data_frame_Life_Stage = Species_data_frame_Life_Stage.loc[Species_data_frame_Life_Stage['ReachName'].isin(Reach_Life_Stage_Presence_Criteria_ReachName)] 
    print('number of reaches/row after Life Stage Presence criteria: ' + str(Species_data_frame_Life_Stage.shape[0]) )
    
    # ------------------------------------------------------------------------------
    #       Identify the Habitat Attributes for the Specific Life Stage
    # ------------------------------------------------------------------------------

    # ------------------ Habitat Attributes for this Life Stage --------------------------------
    Life_Stage_Habitat_Criteria = Species_Life_Stage_Habitat_Attributes.loc[Species_Life_Stage_Habitat_Attributes['Life Stage'].isin(Life_Stage_Habitat_Attributes_vars['variable'][life_stage_val_list])]   
    
    # ----------------- list of Habitat Attributes for this specific Life Stage ---------------
    habitat_attributes_list = Life_Stage_Habitat_Criteria['Habitat Attribute'].values.tolist()
    
    # --------------- Reaches and Habitat Attributes for this Life Stage ------------------
    Life_Stage_Habitat_Criteria_2 = Species_data_frame_Life_Stage.loc[Species_data_frame_Life_Stage['Habitat Attribute'].isin(habitat_attributes_list)]

    # ------------------------------------------------------------------------------
    #      Prepare all the Variables for the Output Table
    # ------------------------------------------------------------------------------
    
    # Example Output Variables (for Spring Chinook - Holding and Maturation):  subbasin,  AU, AU Tier SprChn, 
    #           Spch_Holding_priority (high priority),  ReachName,  SPCH_Holding,  Species, Life_Stage, Habitat_Attribute,  
    #          DRAFT Habitat Attribute Score

    # Update Assessment Unit column name to be identical to Life Stage data frame
    AU_Life_Stage_Priority_Criteria = AU_Life_Stage_Priority_Criteria.rename(columns = {'AU':'Assessment Unit'})
    
    # Append Life Stage Habitat Criteria and AU life Stage Priority data frame to include AU Life Stage Priority
    Life_Stage_Habitat_Criteria_combined = pd.merge(Life_Stage_Habitat_Criteria_2, AU_Life_Stage_Priority_Criteria, how='left',  on = ['Assessment Unit'])
    
    # Append Life Stage Habitat Criteria and AU life Stage Priority data frame to include AU Life Stage Priority
    Life_Stage_Habitat_Criteria_combined = pd.merge(Life_Stage_Habitat_Criteria_combined, Reach_Life_Stage_Presence_Criteria, how='left',  on = ['ReachName'])

    # generate column of species name
    rows_x = Life_Stage_Habitat_Criteria_combined.shape[0]  # get number of rows in Life_Stage_Habitat_Criteria_combined
    col_x = pd.DataFrame({'Species': [species] * rows_x})  # create columns
    Life_Stage_Habitat_Criteria_combined['Species'] = col_x          # add column
    
    # add Life Stage name
    col_x = pd.DataFrame({'Species': [Life_Stage_Priority_vars['variable'][life_stage_val]] * rows_x})  # create columns
    Life_Stage_Habitat_Criteria_combined['Life_Stage'] = col_x          # add column
    

    # ------------------------------------------------------------------------------
    #      Create Output Table
    # ------------------------------------------------------------------------------

    # create output that matches the output table in Access
    Output_data_frame = Life_Stage_Habitat_Criteria_combined[['Species', 'Subbasin_x', 'Assessment Unit','ReachName', 
        'Life_Stage' , 'Habitat Attribute', 'DRAFT Habitat Attribute Score (Currently MIN Score OR NA)', output_column_AU_column,  
           Life_Stage_Priority_vars[variable_name][life_stage_val],   Reach_Life_Stage_Presence_vars[variable_name][life_stage_val] ]]

    Output_data_frame = Output_data_frame.rename(columns={'Subbasin_x':'Subbasin', 
                                      'DRAFT Habitat Attribute Score (Currently MIN Score OR NA)' : 'Individual Habitat Attribute Score'})

    # re-order based on ReachName
    Output_data_frame_X = Output_data_frame.sort_values(by=['ReachName'])
    # Output_data_frame_X.iloc[0:78]    # just can use to output data

        # ------------------------------------------------------------------------------
        #       Generate Columns for Output
        # ------------------------------------------------------------------------------

        #print('completed output for Species: ' + species + "for the Life Stage: " + life_stage)
        #print(' ----------------------------------------------------------------------------------')
        #print("      ")

    Output_data_frame_X.to_excel(output_path, index = False)
    return(Output_data_frame_X)




## ---------------------------------------------------------------------------------------------------------------------------
## Spring Chinook Output Loop
## ---------------------------------------------------------------------------------------------------------------------------

In [559]:
# ------ just a dummy array for index values ------------
index_vals = pd.Series(range(0,7))

species = "Spring Chinook"

Spring_Chinook_Limiting_Factor_Output_All_Stages = pd.DataFrame([])
for i in range(index_vals.shape[0]):
    # -------------- test variables ---------- 
    life_stage = life_stages_all.life_stages[i]
    output_path_x = Output_Spring_Chinook_list[i]

    # --------- Run Function to generate output ----------
    life_stage_output = Generate_Limiting_Factor_Output_Table( life_stage,    species  ,  output_path_x)
    Spring_Chinook_Limiting_Factor_Output_All_Stages = Spring_Chinook_Limiting_Factor_Output_All_Stages.append(life_stage_output)

        
# ------------- Write to Excel --------------
Spring_Chinook_Limiting_Factor_Output_All_Stages.to_excel(output_location_master_spring_chinook, index = False)


------------------- starting output for Species: Spring Chinookfor the Life Stage: Adult Migration
number of reaches/row after AU Life Stage Priority criteria: 0
number of reaches/row after Life Stage Presence criteria: 0
------------------- starting output for Species: Spring Chinookfor the Life Stage: Holding and Maturation
number of reaches/row after AU Life Stage Priority criteria: 155
number of reaches/row after Life Stage Presence criteria: 155
------------------- starting output for Species: Spring Chinookfor the Life Stage: Spawning
number of reaches/row after AU Life Stage Priority criteria: 155
number of reaches/row after Life Stage Presence criteria: 155
------------------- starting output for Species: Spring Chinookfor the Life Stage: Fry
number of reaches/row after AU Life Stage Priority criteria: 102
number of reaches/row after Life Stage Presence criteria: 102
------------------- starting output for Species: Spring Chinookfor the Life Stage: Summer Rearing
number of reac

In [560]:
#  ---------------- SCRIPTS to check out specific reaches ------------------
reach_name = 'Entiat River Lake 01'
# habitat_attribute_scores[habitat_attribute_scores.ReachName.isin([reach_name])]


## ---------------------------------------------------------------------------------------------------------------------------
## Steelhead Output Loop
## ---------------------------------------------------------------------------------------------------------------------------

In [561]:
# ------ just a dummy array for index values ------------
index_vals = pd.Series(range(0,7))

species = "Steelhead"

Steelhead_Limiting_Factor_Output_All_Stages = pd.DataFrame([])
for i in range(index_vals.shape[0]):
    # -------------- test variables ---------- 
    life_stage = life_stages_all.life_stages[i]
    output_path_x = Output_Steelhead_list[i]

    # --------- Run Function to generate output ----------
    life_stage_output = Generate_Limiting_Factor_Output_Table( life_stage,    species  ,  output_path_x)
    Steelhead_Limiting_Factor_Output_All_Stages = Steelhead_Limiting_Factor_Output_All_Stages.append(life_stage_output)

# ------------- Write to Excel --------------
Steelhead_Limiting_Factor_Output_All_Stages.to_excel(output_location_master_steelhead, index = False)


------------------- starting output for Species: Steelheadfor the Life Stage: Adult Migration
number of reaches/row after AU Life Stage Priority criteria: 55
number of reaches/row after Life Stage Presence criteria: 53
------------------- starting output for Species: Steelheadfor the Life Stage: Holding and Maturation
number of reaches/row after AU Life Stage Priority criteria: 0
number of reaches/row after Life Stage Presence criteria: 0
------------------- starting output for Species: Steelheadfor the Life Stage: Spawning
number of reaches/row after AU Life Stage Priority criteria: 48
number of reaches/row after Life Stage Presence criteria: 48
------------------- starting output for Species: Steelheadfor the Life Stage: Fry
number of reaches/row after AU Life Stage Priority criteria: 0
number of reaches/row after Life Stage Presence criteria: 0
------------------- starting output for Species: Steelheadfor the Life Stage: Summer Rearing
number of reaches/row after AU Life Stage Prior

In [562]:
# Steelhead_Limiting_Factor_Output_All_Stages.loc[Steelhead_Limiting_Factor_Output_All_Stages.ReachName == 'Roaring Creek Nason 02']

In [563]:
# ------ just to pring out life stages in column and the priorities in each column (to see which have no "High Priority") ---
for column_x in AU_life_stage_priorities.columns[2:16]:
    print(column_x)
    print(AU_life_stage_priorities[column_x].unique())
    print(" ")
    
# AU life stage priorities without High Priority: SPCH_AdultMigrationLSP, SH_HoldingLSP, SH_FryLSP

SPCH_AdultMigrationLSP
['Life Stage Not Supported' 'Low Priority' 'Missing Data'
 'Medium Priority']
 
SPCH_HoldingLSP
['Life Stage Not Supported' 'Low Priority' 'Medium Priority'
 'Missing Data' 'High Priority']
 
SPCH_SpawnLSP
['Life Stage Not Supported' 'Low Priority' 'Medium Priority'
 'Missing Data' 'High Priority']
 
SPCH_FryLSP
['Life Stage Not Supported' 'Low Priority' 'Medium Priority'
 'High Priority' 'Missing Data']
 
SPCH_SummerRearingLSP
['Life Stage Not Supported' 'Low Priority' 'Medium Priority'
 'High Priority' 'Missing Data' 'Not a Priority at this Time']
 
SPCH_WinterRearingLSP
['Life Stage Not Supported' 'Low Priority' 'Medium Priority'
 'High Priority' 'Missing Data']
 
SPCH_SmoltLSP
['Life Stage Not Supported' 'Low Priority' 'Medium Priority'
 'High Priority' 'Missing Data']
 
SH_AdultMigrationLSP
['Life Stage Not Supported' 'Medium Priority' 'Low Priority'
 'High Priority' 'Not a Priority at this TIme' 'Missing Data']
 
SH_HoldingLSP
['Life Stage Not Supported' 'M

In [508]:
#  ---------------- SCRIPTS to check out specific reaches ------------------
reach_name = 'Nason Creek Lower 01'
# habitat_attribute_scores[habitat_attribute_scores.ReachName.isin([reach_name])]


# ------------------------- Diagnostic test to verifiy outputs are correct -----------------------

## NOTE: A more thorough is output with specific criteria, and manually compare to MASTER excel FOR each species and life stages

#### Steps 1) Go into MASTER_Step2_HabitatAnalysis, then go to "HabitatAttribute_Scores" tab
####       2) in the MASTER... spreadsheet, choose criteria that matches with this output (of the five filters, that tab has all but confinment)
####        ------a) Spring Chinook/Steelhead Reach: Spring.Chinook.Reach  -> "yes"
####        ------b) AU_Rank: AU Rank Sp CHN (for STLHD) -> 1
####        ------c) Cumulative Habitat Attribute Score across habitat attributes for individual reaches Criteria: DRAFT Habitat Attribute Score (Currently MAX Score OR "0" if NA)  -> 5
####      
####       3) copy the "filtered" HabitatAttribute_Scores spreadsheet, paste in a new excel spreadsheet and save
####       3) make sure the criteria in this script is exact as in your filter (set Reach_Life_Stage_Presence to [0, 1], set Life_Stage_Priority to ["Life Stage Not Supported", "Low Priority", "Medium Priority", "High Priority"]  )
####       4) run the script below (with correct path to the saved spreadsheet)

In [37]:
# -------------------------------------------------------------------------------------
#            Reach Names generated from this scrpit RIGHT NOW 
# -------------------------------------------------------------------------------------
# NOTE - use this if you set Reach_Life_Stage_Presence to [0, 1]and Life_Stage_Priority to ["Life Stage Not Supported", "Low Priority", "Medium Priority", "High Priority"]
#           here and have run the script

python_script_output_Steelhead_ReachNames = Steelhead_Limiting_Factor_Output_All_Stages.ReachName.tolist()
python_script_output_Spring_Chinook_ReachNames = Spring_Chinook_Limiting_Factor_Output_All_Stages.ReachName.tolist()
# ----------- sort Reach Names so they are in order ----------
python_script_output_Steelhead_ReachNames.sort()
python_script_output_Spring_Chinook_ReachNames.sort()

In [129]:
# -------------------------------------------------------------------------------------
#            Reach Names generated from this scrpit (saved on 30.Sept.2020 w/ Reach_Life_Stage_Presence to [0, 1], set Life_Stage_Priority to ["Life Stage Not Supported", "Low Priority", "Medium Priority", "High Priority"])
# -------------------------------------------------------------------------------------
# SO use this if you just want to check - since this output was saved earlier

#python_script_output_Steelhead_ReachNames = pd.read_excel('Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/Limiting_Factor,Steelhead,all_Life_Stages,ALL_SCORES_Sept30.xlsx')
#python_script_output_Spring_Chinook_ReachNames = pd.read_excel('Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/Limiting_Factor,Spring_Chinook,all_Life_Stages,ALL_SCORES_Sept30.xlsx')
# ---------------- Just get the Reach Name to list ----------
#python_script_output_Steelhead_ReachNames = python_script_output_Steelhead_ReachNames.ReachName.tolist()
#python_script_output_Spring_Chinook_ReachNames = python_script_output_Spring_Chinook_ReachNames.ReachName.tolist()
#----------- sort Reach Names so they are in order ----------
#python_script_output_Steelhead_ReachNames.sort()
#python_script_output_Spring_Chinook_ReachNames.sort()



In [189]:
# -------------------------------------------------------------------------------------
#            Reach Names copied from MASTER_Step2_HabitatAnalysis
# -------------------------------------------------------------------------------------

# ------------------- Read in Saved Filter data (from MASTER_Step2_HabitatAnalysis excel file) -----------------
LF_Steelhead_filtered = pd.read_excel('Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/LF_Steelhead_FROM_FILTER_IN_MASTER_Step2_HabitatAnalysis_12Oct.xlsx')
LF_Spring_Chinook_filtered = pd.read_excel('Y:/UCRTT/Prioritization/Step 2/Habitat Evaluation/Step2_Prioritization_Python_Output/LF_Spring_Chinook_FROM_FILTER_IN_MASTER_Step2_HabitatAnalysis_12Oct.xlsx')
# ---------------- Just get the Reach Name from filtered MASTER data ----------
MASTER_output_Steelhead_ReachNames = LF_Steelhead_filtered.ReachName.tolist()
MASTER_output_Spring_Chinook_ReachNames = LF_Spring_Chinook_filtered.ReachName.tolist()
# ----------- sort Reach Names so they are in order ----------
MASTER_output_Steelhead_ReachNames.sort()
MASTER_output_Spring_Chinook_ReachNames.sort()

In [155]:
LF_Spring_Chinook_filtered.columns

Index(['ReachName', 'Assessment Unit', 'Subbasin', 'Spring.Chinook.Reach',
       'Steelhead.Reach', 'Bull.Trout.Reach', 'StreamWidthCategory',
       'AU Tier Spring CHN', 'AU Tier Steelhead', 'Habitat Attribute',
       'Reach- Habitat Attribute', 'Life Stages',
       'DataSouce (see Habitat_Data_Raw for data)', 'HabitatAttributeScore1',
       'HabitatAttributeScore2', 'HabitatAttributeScore3',
       'HabitatAttributeScore4',
       'DRAFT Habitat Attribute Score (Currently MIN Score OR NA)', 'Notes'],
      dtype='object')

In [159]:
LF_Steelhead_filtered['DRAFT Habitat Attribute Score (Currently MIN Score OR NA)'].unique()

array([1, 3], dtype=int64)

## --------------------- Script to output IF there are any reach discrepencies
### have three outputs: "MASTER_output_Steelhead_ReachNames identical list - test passed!","Filtered MASTER data has extra reaches: BLANK", OR "Python script output data has extra reaches: BLANK"

In [190]:
# ---------------------------------------------------------------
#            Functions to identify discrepencies
# ---------------------------------------------------------------

def is_identical(list_a, list_b):
    list_a.sort()
    list_b.sort()
    return list_a == list_b


# this function outputs items in list_a that are not in list_b
def get_difference(list_a, list_b):
    list_a.sort()
    list_b.sort()
    return set(list_a)-set(list_b)
 
# did not use this one
l_func = lambda x, y: list((set(x)- set(y))) + list((set(y)- set(x))) 


## --------------------- Run IF loops ---------------------------

# ------------------------- Spring Chinook -----------------------

In [191]:

# -------------------- Identify if lists are the same ------------

if is_identical(MASTER_output_Spring_Chinook_ReachNames, python_script_output_Spring_Chinook_ReachNames):
    print("Two Spring Chinook lists are identical.")
else:
    print("Two Spring Chinook lists are different.")
    print("  ") 
    
# ---------- gets "extra" reaches in MASTER_Step2_HabitatAnalysis ---------------
if(len(get_difference(MASTER_output_Spring_Chinook_ReachNames, python_script_output_Spring_Chinook_ReachNames)) > 0):

    reaches_MASTER = get_difference(MASTER_output_Spring_Chinook_ReachNames, python_script_output_Spring_Chinook_ReachNames)

    print("List of 'extra' MASTER/Original reaches at 'reaches_MASTER', total is: " + str(len(reaches_MASTER)))
    print("Reach Names in MASTER/Original: ")
    print(reaches_MASTER)
print("  ") 

# ---------- gets "extra" reaches generated by python script ---------------
if(len(get_difference(python_script_output_Spring_Chinook_ReachNames, MASTER_output_Spring_Chinook_ReachNames)) > 0):
    reaches_Python = get_difference(python_script_output_Spring_Chinook_ReachNames, MASTER_output_Spring_Chinook_ReachNames)
    print("List of 'extra' Python script reaches at 'reaches_Python', total is: " + str(len(reaches_Python)))
    print("Reach Names in Python Script: ")
    print(reaches_Python)



Two Spring Chinook lists are different.
  
  


In [133]:
len(python_script_output_Spring_Chinook_ReachNames)

1993

In [134]:
len(MASTER_output_Spring_Chinook_ReachNames)

1855

In [ ]:
Steelhead_Limiting_Factor_Output_All_Stages[Steelhead_Limiting_Factor_Output_All_Stages['Assessment Unit'].isin(['Chewuch River-Pearrygin Creek'])]

# ------------------------- Steelhead -----------------------

In [192]:

# -------------------- Identify if lists are the same ------------

if is_identical(MASTER_output_Steelhead_ReachNames, python_script_output_Steelhead_ReachNames):
    print("Two Steelhead lists are identical.")
else:
    print("Two Steelhead lists are different.")
    print("  ") 
    
# ---------- gets "extra" reaches in MASTER_Step2_HabitatAnalysis ---------------
if(len(get_difference(MASTER_output_Steelhead_ReachNames, python_script_output_Steelhead_ReachNames)) > 0):

    reaches_MASTER = get_difference(MASTER_output_Steelhead_ReachNames, python_script_output_Steelhead_ReachNames)

    print("List of 'extra' MASTER/Original reaches at 'reaches_MASTER', total is: " + str(len(reaches_MASTER)))
    print("Reach Names in MASTER/Original: ")
    print(reaches_MASTER)
print("  ") 

# ---------- gets "extra" reaches generated by python script ---------------
if(len(get_difference(python_script_output_Steelhead_ReachNames, MASTER_output_Steelhead_ReachNames)) > 0):
    reaches_Python = get_difference(python_script_output_Steelhead_ReachNames, MASTER_output_Steelhead_ReachNames)
    print("List of 'extra' Python script reaches at 'reaches_Python', total is: " + str(len(reaches_Python)))
    print("Reach Names in Python Script: ")
    print(reaches_Python)



Two Steelhead lists are different.
  
List of 'extra' MASTER/Original reaches at 'reaches_MASTER', total is: 4
Reach Names in MASTER/Original: 
{'Roaring Creek Nason 02', 'Doe Creek 01', 'Roaring Creek Nason 03', 'Scaffold Camp Creek 01'}
  


In [139]:
len(python_script_output_Steelhead_ReachNames)

2389

In [140]:
len(MASTER_output_Steelhead_ReachNames)

2815

In [ ]:
unique(LF_Steelhead_filtered[''])

In [193]:
Steelhead_Limiting_Factor_Output_All_Stages[Steelhead_Limiting_Factor_Output_All_Stages.ReachName == "Doe Creek 01"]

,Species,Subbasin,Assessment Unit,ReachName,Life_Stage,Habitat Attribute,Cumulative Habitat Attribute Score,AU Tier Steelhead,SH_Holding,SH_AdultMigrationLSP,SH_Adult_Migration,Individual Habitat Scores,SH_HoldingLSP,SH_SpawningLSP,SH_Spawning,SH_FryLSP,SH_Fry_Colonization,SH_SummerRearingLSP,SH_Summer_Rearing,SH_WinterRearingLSP,SH_Winter_Rearing,SH_SmoltLSP,SH_Smolt_Emigration


## ---------------------------------------------------------------------------------------------------------------------------

# -
# Step 3: Output For A Life Stage Habitat Quality Assessment
# -

## ---------------------------------------------------------------------------------------------------------------------------

In [564]:
# "Create a new output that takes all the habitat attributes linked to each Life Stage, 
#      then do a cumulative calculation (a habitat function calculation like HQ_score) for each life stage in EACH reach-based 
#      This can bee called “life stage habitat quality assessment”"

In [565]:
# ---------------- Loop that loops through each reach then does the following:
#           1) adds all the habitat data (could just use the habitat quality data)
#           2) look for presence of each life stage
#           3) if the life stage is presence, calculates the habitat quality score

In [566]:
# habitat_quality_scores.columns

In [567]:
# -------------------------------- convert habitat quality scores to numeric --------------
habitat_quality_scores['Temperature- Rearing'] = pd.to_numeric(habitat_quality_scores['Temperature- Rearing'] ,errors='coerce')
habitat_quality_scores['Flow- Summer Base Flow'] = pd.to_numeric(habitat_quality_scores['Flow- Summer Base Flow'] ,errors='coerce')
habitat_quality_scores['Riparian- Structure'] = pd.to_numeric(habitat_quality_scores['Riparian- Structure'] ,errors='coerce')
habitat_quality_scores['Riparian-Disturbance'] = pd.to_numeric(habitat_quality_scores['Riparian-Disturbance'] ,errors='coerce')
habitat_quality_scores['Riparian- Canopy Cover'] = pd.to_numeric(habitat_quality_scores['Riparian- Canopy Cover'] ,errors='coerce')
habitat_quality_scores['Riparian'] = pd.to_numeric(habitat_quality_scores['Riparian'] ,errors='coerce')
habitat_quality_scores['Coarse Substrate'] = pd.to_numeric(habitat_quality_scores['Coarse Substrate'] ,errors='coerce')
habitat_quality_scores['Cover- Wood'] = pd.to_numeric(habitat_quality_scores['Cover- Wood'] ,errors='coerce')
habitat_quality_scores['Pool Quantity& Quality'] = pd.to_numeric(habitat_quality_scores['Pool Quantity& Quality'] ,errors='coerce')
habitat_quality_scores['Off-Channel- Floodplain'] = pd.to_numeric(habitat_quality_scores['Off-Channel- Floodplain'] ,errors='coerce')
habitat_quality_scores['Off-Channel- Side-Channels'] = pd.to_numeric(habitat_quality_scores['Off-Channel- Side-Channels'] ,errors='coerce')
# habitat_quality_scores['Off-Channel'] = pd.to_numeric(habitat_quality_scores['Off-Channel'] ,errors='coerce')
habitat_quality_scores['Channel Stability'] = pd.to_numeric(habitat_quality_scores['Channel Stability'] ,errors='coerce')
habitat_quality_scores['Bank Stability'] = pd.to_numeric(habitat_quality_scores['Bank Stability'] ,errors='coerce')
habitat_quality_scores['Stability'] = pd.to_numeric(habitat_quality_scores['Stability'] ,errors='coerce')

## ---------------------------------------------------------------------------------------------------------------------------
## Loop through reaches, calculate composite score for each Life Stage
##       for both Spring Chinook and Steelhead
## ---------------------------------------------------------------------------------------------------------------------------

In [568]:
# habitat_attribute_lifestage_Spring_Chinook.sort_values(by=['Life Stage'])

In [569]:

# -------------- generate empty data frame ------------
Life_Stage_Habita_Quality_Assessment_Spring_Chinook = pd.DataFrame([])
Life_Stage_Habita_Quality_Assessment_Steelhead = pd.DataFrame([])

# -------------- strip white space from life stage - habitat attribute data ------------
habitat_attribute_lifestage = habitat_attribute_lifestage.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# ------------------- make it so "Off-Channel Side-Channels" matches exactly with column names ----
# column names have "-" between "Side" and "Channels"
habitat_attribute_lifestage = habitat_attribute_lifestage.replace('Off-Channel- Side Channels', 'Off-Channel- Side-Channels')

# ---------------- life stages and habitat attributes for each species -------------------- 
habitat_attribute_lifestage_Spring_Chinook = habitat_attribute_lifestage.loc[habitat_attribute_lifestage.Species == 'Spring Chinook']
habitat_attribute_lifestage_Steelhead = habitat_attribute_lifestage.loc[habitat_attribute_lifestage.Species == 'Steelhead']

# --------------- get last column to pull from habitat_quality_scores data frame ----------
habitat_quality_scores_columns = habitat_quality_scores.columns
habitat_quality_scores_columns = habitat_quality_scores_columns.tolist()
last_column = habitat_quality_scores_columns.index('HQ_Sum')


In [570]:
# ---------------------------------- loop through all the life stages -----------------------------
for reach_x in habitat_quality_scores.ReachName:
    
    #  print('-----------------------:' + reach_x)
    # ------------------- choose row with reach habitat data ------------------------
    habitat_quality_scores_row = habitat_quality_scores.loc[habitat_quality_scores.ReachName == reach_x]
    
    # ------------------------ only get part of row with reach info and habitat data --------------
    habitat_quality_scores_row = habitat_quality_scores_row.iloc[:,0:last_column]
    
    # ----------------------- pull life stage presence data for Spring Chinook and Steelhead ------
    reach_life_stage_presence_Spring_Chinook_row = reach_life_stage_presence_Spring_Chinook.loc[reach_life_stage_presence_Spring_Chinook.ReachName == reach_x]
    reach_life_stage_presence_Steelhead_row = reach_life_stage_presence_Steelhead.loc[reach_life_stage_presence_Steelhead.ReachName == reach_x]

    # --------------------------------------------------------------------------------------------
    #
    #
    #   Loop through each life stage, verify if present in reach, if so, calculate score
    #
    #
    # --------------------------------------------------------------------------------------------

    # --------------------------------------------------------------------------------------------
    #
    #                             Spring Chinook
    #
    # --------------------------------------------------------------------------------------------

    
    # --------------------------------------------------------------------------------------------
    #                    Spring Chinook - Adult Migration
    # --------------------------------------------------------------------------------------------

    if any(reach_life_stage_presence_Spring_Chinook_row.SPCH_Adult_Migration == 1):
               
        life_stage_habitat_attributes = habitat_attribute_lifestage_Spring_Chinook.loc[habitat_attribute_lifestage_Spring_Chinook['Life Stage'] == 'Adult Migration']
        life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']
        
        # -------------- loop through life stage attributes -------------
        hq_sum_x = 0  # composite score 
        hq_tot_x = 0
        for habitat_attribute_x in life_stage_habitat_attributes:
            
            # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
            if any(habitat_quality_scores_row.columns == habitat_attribute_x):

                # --------- convert habitat quality score to float -------
                habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])
                
                # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
                hq_tot_x = hq_tot_x + 1
                
                # ---------if the habitat quality score is numeric (non NaN) --------
                if math.isnan(habitat_score_x) == False:
                    hq_sum_x = hq_sum_x + habitat_score_x

                # ------------ sum of habitat quality scores ----------
                adult_migration_sum = hq_sum_x
                # ------------ maximum potential score ----------
                adult_migration_max = (hq_tot_x * 5)
                # -------- percent functional of habitat for this life stage ----------
                adult_migration_prcnt = hq_sum_x / adult_migration_max
                
     #       else:
     #           adult_migration_sum = 'NA'
     #           adult_migration_max = 'NA'
     #           adult_migration_prcnt = 'NA'

    else:
        adult_migration_sum = 'NA'
        adult_migration_max = 'NA'
        adult_migration_prcnt = 'NA'

    # --------------------------------------------------------------------------------------------
    #                    Spring Chinook - Holding
    # --------------------------------------------------------------------------------------------

    if any(reach_life_stage_presence_Spring_Chinook_row.SPCH_Holding == 1):
    
        life_stage_habitat_attributes = habitat_attribute_lifestage_Spring_Chinook.loc[habitat_attribute_lifestage_Spring_Chinook['Life Stage'] == 'Holding and Maturation']
        life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']

        # -------------- loop through life stage attributes -------------
        hq_sum_x = 0  # composite score 
        hq_tot_x = 0
        for habitat_attribute_x in life_stage_habitat_attributes:

            # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
            if any(habitat_quality_scores_row.columns == habitat_attribute_x):

                # --------- convert habitat quality score to float -------
                habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])

                # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
                hq_tot_x = hq_tot_x + 1

                # ---------if the habitat quality score is numeric (non NaN) --------
                if math.isnan(habitat_score_x) == False:
                    hq_sum_x = hq_sum_x + habitat_score_x

                # ------------ sum of habitat quality scores ----------
                holding_sum = hq_sum_x
                # ------------ maximum potential score ----------
                holding_max = (hq_tot_x * 5)
                # -------- percent functional of habitat for this life stage ----------
                holding_prcnt = hq_sum_x / holding_max
        
      #      else:
      #          holding_sum = 'NA'
      #          holding_max = 'NA'
      #          holding_prcnt = 'NA'  
                
    else:
        holding_sum = 'NA'
        holding_max = 'NA'
        holding_prcnt = 'NA'

    # --------------------------------------------------------------------------------------------
    #                    Spring Chinook - Spawning
    # --------------------------------------------------------------------------------------------

    if any(reach_life_stage_presence_Spring_Chinook_row.SPCH_Spawning == 1):
    
        life_stage_habitat_attributes = habitat_attribute_lifestage_Spring_Chinook.loc[habitat_attribute_lifestage_Spring_Chinook['Life Stage'] == 'Spawning and Incubation']
        life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']

        # -------------- loop through life stage attributes -------------
        hq_sum_x = 0  # composite score 
        hq_tot_x = 0
        for habitat_attribute_x in life_stage_habitat_attributes:

            # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
            if any(habitat_quality_scores_row.columns == habitat_attribute_x):

                # --------- convert habitat quality score to float -------
                habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])

                # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
                hq_tot_x = hq_tot_x + 1

                # ---------if the habitat quality score is numeric (non NaN) --------
                if math.isnan(habitat_score_x) == False:
                    hq_sum_x = hq_sum_x + habitat_score_x

                # ------------ sum of habitat quality scores ----------
                spawning_sum = hq_sum_x
                # ------------ maximum potential score ----------
                spawning_max = (hq_tot_x * 5)
                # -------- percent functional of habitat for this life stage ----------
                spawning_prcnt = hq_sum_x / spawning_max
                
      #      else:
      #          spawning_sum = 'NA'
      #          spawning_max = 'NA'
      #          spawning_prcnt = 'NA'

    else:
        spawning_sum = 'NA'
        spawning_max = 'NA'
        spawning_prcnt = 'NA'
        
    # --------------------------------------------------------------------------------------------
    #                    Spring Chinook - Fry Colonization
    # --------------------------------------------------------------------------------------------

    if any(reach_life_stage_presence_Spring_Chinook_row.SPCH_Fry_Colonization == 1):
    
        life_stage_habitat_attributes = habitat_attribute_lifestage_Spring_Chinook.loc[habitat_attribute_lifestage_Spring_Chinook['Life Stage'] == 'Fry']
        life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']

        # -------------- loop through life stage attributes -------------
        hq_sum_x = 0  # composite score 
        hq_tot_x = 0
        for habitat_attribute_x in life_stage_habitat_attributes:

            # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
            if any(habitat_quality_scores_row.columns == habitat_attribute_x):

                # --------- convert habitat quality score to float -------
                habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])

                # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
                hq_tot_x = hq_tot_x + 1

                # ---------if the habitat quality score is numeric (non NaN) --------
                if math.isnan(habitat_score_x) == False:
                    hq_sum_x = hq_sum_x + habitat_score_x

                # ------------ sum of habitat quality scores ----------
                fry_sum = hq_sum_x
                # ------------ maximum potential score ----------
                fry_max = (hq_tot_x * 5)
                # -------- percent functional of habitat for this life stage ----------
                fry_prcnt = hq_sum_x / fry_max

      #      else:
      #          fry_sum = 'NA'
      #          fry_max = 'NA'
      #          fry_prcnt = 'NA'

    else:
        fry_sum = 'NA'
        fry_max = 'NA'
        fry_prcnt = 'NA'

    # --------------------------------------------------------------------------------------------
    #                    Spring Chinook - Summer Rearing
    # --------------------------------------------------------------------------------------------

    if any(reach_life_stage_presence_Spring_Chinook_row.SPCH_Summer_Rearing == 1):
    
        life_stage_habitat_attributes = habitat_attribute_lifestage_Spring_Chinook.loc[habitat_attribute_lifestage_Spring_Chinook['Life Stage'] == 'Summer Rearing']
        life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']

        # -------------- loop through life stage attributes -------------
        hq_sum_x = 0  # composite score 
        hq_tot_x = 0
        for habitat_attribute_x in life_stage_habitat_attributes:

            # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
            if any(habitat_quality_scores_row.columns == habitat_attribute_x):

                # --------- convert habitat quality score to float -------
                habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])

                # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
                hq_tot_x = hq_tot_x + 1

                # ---------if the habitat quality score is numeric (non NaN) --------
                if math.isnan(habitat_score_x) == False:
                    hq_sum_x = hq_sum_x + habitat_score_x

                # ------------ sum of habitat quality scores ----------
                summer_rearing_sum = hq_sum_x
                # ------------ maximum potential score ----------
                summer_rearing_max = (hq_tot_x * 5)
                # -------- percent functional of habitat for this life stage ----------
                summer_rearing_prcnt = hq_sum_x / summer_rearing_max

      #      else:
      #          summer_rearing_sum = 'NA'
      #          summer_rearing_max = 'NA'
      #          summer_rearing_prcnt = 'NA'
        
    else:
        summer_rearing_sum = 'NA'
        summer_rearing_max = 'NA'
        summer_rearing_prcnt = 'NA'
        
    # --------------------------------------------------------------------------------------------
    #                    Spring Chinook - Winter Rearing
    # --------------------------------------------------------------------------------------------

    if any(reach_life_stage_presence_Spring_Chinook_row.SPCH_Winter_Rearing == 1):
    
        life_stage_habitat_attributes = habitat_attribute_lifestage_Spring_Chinook.loc[habitat_attribute_lifestage_Spring_Chinook['Life Stage'] == 'Winter Rearing']
        life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']

        # -------------- loop through life stage attributes -------------
        hq_sum_x = 0  # composite score 
        hq_tot_x = 0
        for habitat_attribute_x in life_stage_habitat_attributes:

            # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
            if any(habitat_quality_scores_row.columns == habitat_attribute_x):

                # --------- convert habitat quality score to float -------
                habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])

                # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
                hq_tot_x = hq_tot_x + 1

                # ---------if the habitat quality score is numeric (non NaN) --------
                if math.isnan(habitat_score_x) == False:
                    hq_sum_x = hq_sum_x + habitat_score_x

                # ------------ sum of habitat quality scores ----------
                winter_rearing_sum = hq_sum_x
                # ------------ maximum potential score ----------
                winter_rearing_max = (hq_tot_x * 5)
                # -------- percent functional of habitat for this life stage ----------
                winter_rearing_prcnt = hq_sum_x / winter_rearing_max

  #      else:
  #          winter_rearing_sum = 'NA'
  #          winter_rearing_max = 'NA'
  #          winter_rearing_prcnt = 'NA'
        
    else:
        winter_rearing_sum = 'NA'
        winter_rearing_max = 'NA'
        winter_rearing_prcnt = 'NA'
        
    # --------------------------------------------------------------------------------------------
    #                    Spring Chinook - Smolt Emigration
    # --------------------------------------------------------------------------------------------
    

    if any(reach_life_stage_presence_Spring_Chinook_row.SPCH_Smolt_Emigration == 1):
    
        life_stage_habitat_attributes = habitat_attribute_lifestage_Spring_Chinook.loc[habitat_attribute_lifestage_Spring_Chinook['Life Stage'] == 'Smolt Outmigration']
        life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']

        # -------------- loop through life stage attributes -------------
        hq_sum_x = 0  # composite score 
        hq_tot_x = 0
        for habitat_attribute_x in life_stage_habitat_attributes:

            # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
            if any(habitat_quality_scores_row.columns == habitat_attribute_x):

                # --------- convert habitat quality score to float -------
                habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])

                # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
                hq_tot_x = hq_tot_x + 1

                # ---------if the habitat quality score is numeric (non NaN) --------
                if math.isnan(habitat_score_x) == False:
                    hq_sum_x = hq_sum_x + habitat_score_x

                # ------------ sum of habitat quality scores ----------
                smolt_emigration_sum = hq_sum_x
                # ------------ maximum potential score ----------
                smolt_emigration_max = (hq_tot_x * 5)
                # -------- percent functional of habitat for this life stage ----------
                smolt_emigration_prcnt = hq_sum_x / smolt_emigration_max

     #       else:
     #           smolt_emigration_sum = 'NA'
     #           smolt_emigration_max = 'NA'
     #           smolt_emigration_prcnt = 'NA'
    else:
        smolt_emigration_sum = 'NA'
        smolt_emigration_max = 'NA'
        smolt_emigration_prcnt = 'NA'
       
    
    # --------------------------------------------------------------------------------------------
    #            Add Life Stage Presence/Absence data to row
    # --------------------------------------------------------------------------------------------
    life_stage_presence = reach_life_stage_presence_Spring_Chinook_row[['SPCH_Adult_Migration','SPCH_Holding','SPCH_Spawning',
                    'SPCH_Fry_Colonization','SPCH_Summer_Rearing','SPCH_Winter_Rearing','SPCH_Smolt_Emigration']]
    life_stage_presence = life_stage_presence.set_index(habitat_quality_scores_row.index)

    habitat_quality_scores_row_Spring_Chinook = habitat_quality_scores_row.join(  life_stage_presence )

    
    # --------------------------------------------------------------------------------------------
    #            Combine all the Spring Chinook
    # --------------------------------------------------------------------------------------------

    #habitat_quality_row_x2 = pd.DataFrame([adult_migration_sum, adult_migration_prcnt])
    habitat_quality_scores_row_Spring_Chinook = habitat_quality_scores_row_Spring_Chinook.join(pd.DataFrame(
    {
        'adult_migration_sum': adult_migration_sum,
        'adult_migration_max': adult_migration_max,
        'adult_migration_prcnt': adult_migration_prcnt,
        'holding_sum': holding_sum,
        'holding_max': holding_max,
        'holding_prcnt': holding_prcnt,
        'spawning_sum': spawning_sum,
        'spawning_max': spawning_max,
        'spawning_prcnt': spawning_prcnt,
        'fry_sum': fry_sum,
        'fry_max': fry_max,
        'fry_prcnt': fry_prcnt,
        'summer_rearing_sum': summer_rearing_sum,
        'summer_rearing_max': summer_rearing_max,
        'summer_rearing_prcnt': summer_rearing_prcnt,
        'winter_rearing_sum': winter_rearing_sum,
        'winter_rearing_max': winter_rearing_max,
        'winter_rearing_prcnt': winter_rearing_prcnt,
        'smolt_emigration_sum': smolt_emigration_sum,
        'smolt_emigration_max': smolt_emigration_max,
        'smolt_emigration_prcnt': smolt_emigration_prcnt,
        
    }, index=habitat_quality_scores_row_Spring_Chinook.index ), how='right' )

    Life_Stage_Habita_Quality_Assessment_Spring_Chinook = Life_Stage_Habita_Quality_Assessment_Spring_Chinook.append(habitat_quality_scores_row_Spring_Chinook)
    
    
    # --------------------------------------------------------------------------------------------
    #
    #                             Steelhead
    #
    # --------------------------------------------------------------------------------------------

    
    # --------------------------------------------------------------------------------------------
    #                    Steelhead - Adult Migration
    # --------------------------------------------------------------------------------------------

    if any(reach_life_stage_presence_Steelhead_row.SH_Adult_Migration == 1):
               
        life_stage_habitat_attributes = habitat_attribute_lifestage_Steelhead.loc[habitat_attribute_lifestage_Steelhead['Life Stage'] == 'Adult Migration']
        life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']
        
        # -------------- loop through life stage attributes -------------
        hq_sum_x = 0  # composite score 
        hq_tot_x = 0
        for habitat_attribute_x in life_stage_habitat_attributes:
            
            # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
            if any(habitat_quality_scores_row.columns == habitat_attribute_x):

                # --------- convert habitat quality score to float -------
                habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])
                
                # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
                hq_tot_x = hq_tot_x + 1
                
                # ---------if the habitat quality score is numeric (non NaN) --------
                if math.isnan(habitat_score_x) == False:
                    hq_sum_x = hq_sum_x + habitat_score_x

                # ------------ sum of habitat quality scores ----------
                adult_migration_sum = hq_sum_x
                # ------------ maximum potential score ----------
                adult_migration_max = (hq_tot_x * 5)
                # -------- percent functional of habitat for this life stage ----------
                adult_migration_prcnt = hq_sum_x / adult_migration_max
                
        #    else:
        #        adult_migration_sum = 'NA'
        #        adult_migration_max = 'NA'
        #        adult_migration_prcnt = 'NA'

    else:
        adult_migration_sum = 'NA'
        adult_migration_max = 'NA'
        adult_migration_prcnt = 'NA'

    # --------------------------------------------------------------------------------------------
    #                    Steelhead - Holding
    # --------------------------------------------------------------------------------------------

    if any(reach_life_stage_presence_Steelhead_row.SH_Holding == 1):
    
        life_stage_habitat_attributes = habitat_attribute_lifestage_Steelhead.loc[habitat_attribute_lifestage_Steelhead['Life Stage'] == 'Holding and Maturation']
        life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']

        # -------------- loop through life stage attributes -------------
        hq_sum_x = 0  # composite score 
        hq_tot_x = 0
        for habitat_attribute_x in life_stage_habitat_attributes:

            # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
            if any(habitat_quality_scores_row.columns == habitat_attribute_x):

                # --------- convert habitat quality score to float -------
                habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])

                # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
                hq_tot_x = hq_tot_x + 1

                # ---------if the habitat quality score is numeric (non NaN) --------
                if math.isnan(habitat_score_x) == False:
                    hq_sum_x = hq_sum_x + habitat_score_x

                # ------------ sum of habitat quality scores ----------
                holding_sum = hq_sum_x
                # ------------ maximum potential score ----------
                holding_max = (hq_tot_x * 5)
                # -------- percent functional of habitat for this life stage ----------
                holding_prcnt = hq_sum_x / holding_max
        
      #      else:
      #          holding_sum = 'NA'
      #          holding_max = 'NA'
      #          holding_prcnt = 'NA'  
                
    else:
        holding_sum = 'NA'
        holding_max = 'NA'
        holding_prcnt = 'NA'

    # --------------------------------------------------------------------------------------------
    #                    Steelhead - Spawning
    # --------------------------------------------------------------------------------------------

    if any(reach_life_stage_presence_Steelhead_row.SH_Spawning == 1):
    
        life_stage_habitat_attributes = habitat_attribute_lifestage_Steelhead.loc[habitat_attribute_lifestage_Steelhead['Life Stage'] == 'Spawning and Incubation']
        life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']

        # -------------- loop through life stage attributes -------------
        hq_sum_x = 0  # composite score 
        hq_tot_x = 0
        for habitat_attribute_x in life_stage_habitat_attributes:

            # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
            if any(habitat_quality_scores_row.columns == habitat_attribute_x):

                # --------- convert habitat quality score to float -------
                habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])

                # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
                hq_tot_x = hq_tot_x + 1

                # ---------if the habitat quality score is numeric (non NaN) --------
                if math.isnan(habitat_score_x) == False:
                    hq_sum_x = hq_sum_x + habitat_score_x

                # ------------ sum of habitat quality scores ----------
                spawning_sum = hq_sum_x
                # ------------ maximum potential score ----------
                spawning_max = (hq_tot_x * 5)
                # -------- percent functional of habitat for this life stage ----------
                spawning_prcnt = hq_sum_x / spawning_max
                
      #      else:
      #          spawning_sum = 'NA'
      #          spawning_max = 'NA'
      #          spawning_prcnt = 'NA'

    else:
        spawning_sum = 'NA'
        spawning_max = 'NA'
        spawning_prcnt = 'NA'
        
    # --------------------------------------------------------------------------------------------
    #                    Steelhead - Fry Colonization
    # --------------------------------------------------------------------------------------------

    if any(reach_life_stage_presence_Steelhead_row.SH_Fry_Colonization == 1):
    
        life_stage_habitat_attributes = habitat_attribute_lifestage_Steelhead.loc[habitat_attribute_lifestage_Steelhead['Life Stage'] == 'Fry']
        life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']

        # -------------- loop through life stage attributes -------------
        hq_sum_x = 0  # composite score 
        hq_tot_x = 0
        for habitat_attribute_x in life_stage_habitat_attributes:

            # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
            if any(habitat_quality_scores_row.columns == habitat_attribute_x):

                # --------- convert habitat quality score to float -------
                habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])

                # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
                hq_tot_x = hq_tot_x + 1

                # ---------if the habitat quality score is numeric (non NaN) --------
                if math.isnan(habitat_score_x) == False:
                    hq_sum_x = hq_sum_x + habitat_score_x

                # ------------ sum of habitat quality scores ----------
                fry_sum = hq_sum_x
                # ------------ maximum potential score ----------
                fry_max = (hq_tot_x * 5)
                # -------- percent functional of habitat for this life stage ----------
                fry_prcnt = hq_sum_x / fry_max

      #      else:
      #          fry_sum = 'NA'
      #          fry_max = 'NA'
      #          fry_prcnt = 'NA'

    else:
        fry_sum = 'NA'
        fry_max = 'NA'
        fry_prcnt = 'NA'

    # --------------------------------------------------------------------------------------------
    #                    Steelhead - Summer Rearing
    # --------------------------------------------------------------------------------------------

    if any(reach_life_stage_presence_Steelhead_row.SH_Summer_Rearing == 1):
    
        life_stage_habitat_attributes = habitat_attribute_lifestage_Steelhead.loc[habitat_attribute_lifestage_Steelhead['Life Stage'] == 'Summer Rearing']
        life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']

        # -------------- loop through life stage attributes -------------
        hq_sum_x = 0  # composite score 
        hq_tot_x = 0
        for habitat_attribute_x in life_stage_habitat_attributes:

            # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
            if any(habitat_quality_scores_row.columns == habitat_attribute_x):

                # --------- convert habitat quality score to float -------
                habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])

                # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
                hq_tot_x = hq_tot_x + 1

                # ---------if the habitat quality score is numeric (non NaN) --------
                if math.isnan(habitat_score_x) == False:
                    hq_sum_x = hq_sum_x + habitat_score_x

                # ------------ sum of habitat quality scores ----------
                summer_rearing_sum = hq_sum_x
                # ------------ maximum potential score ----------
                summer_rearing_max = (hq_tot_x * 5)
                # -------- percent functional of habitat for this life stage ----------
                summer_rearing_prcnt = hq_sum_x / summer_rearing_max

         #   else:
         #       summer_rearing_sum = 'NA'
         #       summer_rearing_max = 'NA'
         #       summer_rearing_prcnt = 'NA'
        
    else:
        summer_rearing_sum = 'NA'
        summer_rearing_max = 'NA'
        summer_rearing_prcnt = 'NA'
        
    # --------------------------------------------------------------------------------------------
    #                    Steelhead - Winter Rearing
    # --------------------------------------------------------------------------------------------

    if any(reach_life_stage_presence_Steelhead_row.SH_Winter_Rearing == 1):
    
        life_stage_habitat_attributes = habitat_attribute_lifestage_Steelhead.loc[habitat_attribute_lifestage_Steelhead['Life Stage'] == 'Winter Rearing']
        life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']

        # -------------- loop through life stage attributes -------------
        hq_sum_x = 0  # composite score 
        hq_tot_x = 0
        for habitat_attribute_x in life_stage_habitat_attributes:

            # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
            if any(habitat_quality_scores_row.columns == habitat_attribute_x):

                # --------- convert habitat quality score to float -------
                habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])

                # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
                hq_tot_x = hq_tot_x + 1

                # ---------if the habitat quality score is numeric (non NaN) --------
                if math.isnan(habitat_score_x) == False:
                    hq_sum_x = hq_sum_x + habitat_score_x

                # ------------ sum of habitat quality scores ----------
                winter_rearing_sum = hq_sum_x
                # ------------ maximum potential score ----------
                winter_rearing_max = (hq_tot_x * 5)
                # -------- percent functional of habitat for this life stage ----------
                winter_rearing_prcnt = hq_sum_x / winter_rearing_max

        #else:
         #   winter_rearing_sum = 'NA'
         #   winter_rearing_max = 'NA'
         #   winter_rearing_prcnt = 'NA'
        
    else:
        winter_rearing_sum = 'NA'
        winter_rearing_max = 'NA'
        winter_rearing_prcnt = 'NA'
        
    # --------------------------------------------------------------------------------------------
    #                    Steelhead - Smolt Emigration
    # --------------------------------------------------------------------------------------------
    

    if any(reach_life_stage_presence_Steelhead_row.SH_Smolt_Emigration == 1):
    
        life_stage_habitat_attributes = habitat_attribute_lifestage_Steelhead.loc[habitat_attribute_lifestage_Steelhead['Life Stage'] == 'Smolt Outmigration']
        life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']

        # -------------- loop through life stage attributes -------------
        hq_sum_x = 0  # composite score 
        hq_tot_x = 0
        for habitat_attribute_x in life_stage_habitat_attributes:

            # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
            if any(habitat_quality_scores_row.columns == habitat_attribute_x):

                # --------- convert habitat quality score to float -------
                habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])

                # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
                hq_tot_x = hq_tot_x + 1

                # ---------if the habitat quality score is numeric (non NaN) --------
                if math.isnan(habitat_score_x) == False:
                    hq_sum_x = hq_sum_x + habitat_score_x

                # ------------ sum of habitat quality scores ----------
                smolt_emigration_sum = hq_sum_x
                # ------------ maximum potential score ----------
                smolt_emigration_max = (hq_tot_x * 5)
                # -------- percent functional of habitat for this life stage ----------
                smolt_emigration_prcnt = hq_sum_x / smolt_emigration_max

            #else:
                #smolt_emigration_sum = 'NA'
                #smolt_emigration_max = 'NA'
                #smolt_emigration_prcnt = 'NA'
    else:
        smolt_emigration_sum = 'NA'
        smolt_emigration_max = 'NA'
        smolt_emigration_prcnt = 'NA'
        
    # --------------------------------------------------------------------------------------------
    #            Add Life Stage Presence/Absence data to row
    # --------------------------------------------------------------------------------------------
    life_stage_presence = reach_life_stage_presence_Steelhead_row[['SH_Adult_Migration','SH_Holding','SH_Spawning',
                    'SH_Fry_Colonization','SH_Summer_Rearing','SH_Winter_Rearing','SH_Smolt_Emigration']]
    life_stage_presence = life_stage_presence.set_index(habitat_quality_scores_row.index)

    habitat_quality_scores_row_Steelhead =  habitat_quality_scores_row.join(  life_stage_presence )
    
    # --------------------------------------------------------------------------------------------
    #            Combine all the Steelhead
    # --------------------------------------------------------------------------------------------

    #habitat_quality_row_x2 = pd.DataFrame([adult_migration_sum, adult_migration_prcnt])
    habitat_quality_scores_row_Steelhead = habitat_quality_scores_row_Steelhead.join(pd.DataFrame(
    {
        'adult_migration_sum': adult_migration_sum,
        'adult_migration_max': adult_migration_max,
        'adult_migration_prcnt': adult_migration_prcnt,
        'holding_sum': holding_sum,
        'holding_max': holding_max,
        'holding_prcnt': holding_prcnt,
        'spawning_sum': spawning_sum,
        'spawning_max': spawning_max,
        'spawning_prcnt': spawning_prcnt,
        'fry_sum': fry_sum,
        'fry_max': fry_max,
        'fry_prcnt': fry_prcnt,
        'summer_rearing_sum': summer_rearing_sum,
        'summer_rearing_max': summer_rearing_max,
        'summer_rearing_prcnt': summer_rearing_prcnt,
        'winter_rearing_sum': winter_rearing_sum,
        'winter_rearing_max': winter_rearing_max,
        'winter_rearing_prcnt': winter_rearing_prcnt,
        'smolt_emigration_sum': smolt_emigration_sum,
        'smolt_emigration_max': smolt_emigration_max,
        'smolt_emigration_prcnt': smolt_emigration_prcnt,
        
    }, index=habitat_quality_scores_row_Steelhead.index  ), how='right' )

    Life_Stage_Habita_Quality_Assessment_Steelhead = Life_Stage_Habita_Quality_Assessment_Steelhead.append(habitat_quality_scores_row_Steelhead)
    


## ---------------------------------------------------------------------------------------------------------------------------
## Export to Excel
## ---------------------------------------------------------------------------------------------------------------------------

In [571]:
Life_Stage_Habita_Quality_Assessment_Steelhead.to_excel(output_location_life_stage_assessment_steelhead, index = False)
Life_Stage_Habita_Quality_Assessment_Spring_Chinook.to_excel(output_location_life_stage_assessment_spring_chinook, index = False)

In [572]:
Life_Stage_Habita_Quality_Assessment_Steelhead.shape

(739, 52)

In [573]:
# ------------------- JUST for testing code ---------------
reach_x = 'Alder Creek 01'

# ------------------- choose row with reach habitat data ------------------------
habitat_quality_scores_row = habitat_quality_scores.loc[habitat_quality_scores.ReachName == reach_x]

# ------------------------ only get part of row with reach info and habitat data --------------
habitat_quality_scores_row = habitat_quality_scores_row.iloc[:,0:last_column]

# ----------------------- pull life stage presence data for Spring Chinook and Steelhead ------
reach_life_stage_presence_Spring_Chinook_row = reach_life_stage_presence_Spring_Chinook.loc[reach_life_stage_presence_Spring_Chinook.ReachName == reach_x]
reach_life_stage_presence_Steelhead_row = reach_life_stage_presence_Steelhead.loc[reach_life_stage_presence_Steelhead.ReachName == reach_x]


In [574]:

# --------------------------------------------------------------------------------------------
#
#
#   Loop through each life stage, verify if present in reach, if so, calculate score
#
#
# --------------------------------------------------------------------------------------------

# --------------------------------------------------------------------------------------------
#
#                             Spring Chinook
#
# --------------------------------------------------------------------------------------------


# --------------------------------------------------------------------------------------------
#                    Spring Chinook - Adult Migration
# --------------------------------------------------------------------------------------------

if any(reach_life_stage_presence_Spring_Chinook_row.SPCH_Adult_Migration == 1):

    life_stage_habitat_attributes = habitat_attribute_lifestage_Spring_Chinook.loc[habitat_attribute_lifestage_Spring_Chinook['Life Stage'] == 'Adult Migration']
    life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']

    # -------------- loop through life stage attributes -------------
    hq_sum_x = 0  # composite score 
    hq_tot_x = 0
    for habitat_attribute_x in life_stage_habitat_attributes:

        # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
        if any(habitat_quality_scores_row.columns == habitat_attribute_x):

            # --------- convert habitat quality score to float -------
            habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])

            # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
            hq_tot_x = hq_tot_x + 1

            # ---------if the habitat quality score is numeric (non NaN) --------
            if math.isnan(habitat_score_x) == False:
                hq_sum_x = hq_sum_x + habitat_score_x

            # ------------ sum of habitat quality scores ----------
            adult_migration_sum = hq_sum_x
            # ------------ maximum potential score ----------
            adult_migration_max = (hq_tot_x * 5)
            # -------- percent functional of habitat for this life stage ----------
            adult_migration_prcnt = hq_sum_x / adult_migration_max

        else:
            adult_migration_sum = 'NA'
            adult_migration_max = 'NA'
            adult_migration_prcnt = 'NA'

else:
    adult_migration_sum = 'NA'
    adult_migration_max = 'NA'
    adult_migration_prcnt = 'NA'

# --------------------------------------------------------------------------------------------
#                    Spring Chinook - Holding
# --------------------------------------------------------------------------------------------

if any(reach_life_stage_presence_Spring_Chinook_row.SPCH_Holding == 1):

    life_stage_habitat_attributes = habitat_attribute_lifestage_Spring_Chinook.loc[habitat_attribute_lifestage_Spring_Chinook['Life Stage'] == 'Holding and Maturation']
    life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']

    # -------------- loop through life stage attributes -------------
    hq_sum_x = 0  # composite score 
    hq_tot_x = 0
    for habitat_attribute_x in life_stage_habitat_attributes:

        #print('habitat_attribute_x: ' + habitat_attribute_x)
        
        # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
        if any(habitat_quality_scores_row.columns == habitat_attribute_x):
            #print('habitat_quality_scores_row[habitat_attribute_x]: ' )
           # print(habitat_quality_scores_row[habitat_attribute_x])
            # --------- convert habitat quality score to float -------
            habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])

            # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
            hq_tot_x = hq_tot_x + 1

            # ---------if the habitat quality score is numeric (non NaN) --------
            if math.isnan(habitat_score_x) == False:
                
                #print('-------------------------------------in math loop-------------------------')
                #print('habitat_score_x: ')
                #print(habitat_score_x)
                #print('hq_sum_x: ')
                #print(hq_sum_x)
                hq_sum_x = hq_sum_x + habitat_score_x
                #print('habitat_score_x: ')
                #print(habitat_score_x)
                #print('hq_sum_x: ')
                #print(hq_sum_x)

            # ------------ sum of habitat quality scores ----------
            holding_sum = hq_sum_x
            # ------------ maximum potential score ----------
            holding_max = (hq_tot_x * 5)
            # -------- percent functional of habitat for this life stage ----------
            holding_prcnt = hq_sum_x / holding_max

        #else:
           # holding_sum = 'NA'
            #holding_max = 'NA'
           # holding_prcnt = 'NA'  

else:
    holding_sum = 'NA'
    holding_max = 'NA'
    holding_prcnt = 'NA'

print('HOLDING OUTPUT ------------:')
print(holding_sum)
print(holding_max)
print(holding_prcnt)
    
# --------------------------------------------------------------------------------------------
#                    Spring Chinook - Spawning
# --------------------------------------------------------------------------------------------

if any(reach_life_stage_presence_Spring_Chinook_row.SPCH_Spawning == 1):

    life_stage_habitat_attributes = habitat_attribute_lifestage_Spring_Chinook.loc[habitat_attribute_lifestage_Spring_Chinook['Life Stage'] == 'Spawning and Incubation']
    life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']

    # -------------- loop through life stage attributes -------------
    hq_sum_x = 0  # composite score 
    hq_tot_x = 0
    for habitat_attribute_x in life_stage_habitat_attributes:
        
        print('-------------------------------habitat_attribute_x------------------: ' + habitat_attribute_x)

        # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
        if any(habitat_quality_scores_row.columns == habitat_attribute_x):
            print('habitat_quality_scores_row[habitat_attribute_x]: ' )
            print(habitat_quality_scores_row[habitat_attribute_x])
            # --------- convert habitat quality score to float -------
            habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])

            # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
            hq_tot_x = hq_tot_x + 1

            # ---------if the habitat quality score is numeric (non NaN) --------
            if math.isnan(habitat_score_x) == False:
                print('------in math loop-----------')
                print('habitat_score_x: ')
                print(habitat_score_x)
                print('hq_sum_x: ')
                print(hq_sum_x)
                hq_sum_x = hq_sum_x + habitat_score_x


            # ------------ sum of habitat quality scores ----------
            print('habitat_score_x: ')
            print(habitat_score_x)
            print('hq_sum_x: ')
            print(hq_sum_x)

            # ------------ sum of habitat quality scores ----------
            spawning_sum = hq_sum_x
            # ------------ maximum potential score ----------
            spawning_max = (hq_tot_x * 5)
            # -------- percent functional of habitat for this life stage ----------
            spawning_prcnt = hq_sum_x / spawning_max

        else:
            spawning_sum = 'NA'
            spawning_max = 'NA'
            spawning_prcnt = 'NA'

else:
    spawning_sum = 'NA'
    spawning_max = 'NA'
    spawning_prcnt = 'NA'

# --------------------------------------------------------------------------------------------
#                    Spring Chinook - Fry Colonization
# --------------------------------------------------------------------------------------------

if any(reach_life_stage_presence_Spring_Chinook_row.SPCH_Fry_Colonization == 1):

    life_stage_habitat_attributes = habitat_attribute_lifestage_Spring_Chinook.loc[habitat_attribute_lifestage_Spring_Chinook['Life Stage'] == 'Fry']
    life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']

    # -------------- loop through life stage attributes -------------
    hq_sum_x = 0  # composite score 
    hq_tot_x = 0
    for habitat_attribute_x in life_stage_habitat_attributes:

        # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
        if any(habitat_quality_scores_row.columns == habitat_attribute_x):

            # --------- convert habitat quality score to float -------
            habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])

            # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
            hq_tot_x = hq_tot_x + 1

            # ---------if the habitat quality score is numeric (non NaN) --------
            if math.isnan(habitat_score_x) == False:
                hq_sum_x = hq_sum_x + habitat_score_x

            # ------------ sum of habitat quality scores ----------
            fry_sum = hq_sum_x
            # ------------ maximum potential score ----------
            fry_max = (hq_tot_x * 5)
            # -------- percent functional of habitat for this life stage ----------
            fry_prcnt = hq_sum_x / fry_max

        else:
            fry_sum = 'NA'
            fry_max = 'NA'
            fry_prcnt = 'NA'

else:
    fry_sum = 'NA'
    fry_max = 'NA'
    fry_prcnt = 'NA'

# --------------------------------------------------------------------------------------------
#                    Spring Chinook - Summer Rearing
# --------------------------------------------------------------------------------------------

if any(reach_life_stage_presence_Spring_Chinook_row.SPCH_Summer_Rearing == 1):

    life_stage_habitat_attributes = habitat_attribute_lifestage_Spring_Chinook.loc[habitat_attribute_lifestage_Spring_Chinook['Life Stage'] == 'Summer Rearing']
    life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']

    # -------------- loop through life stage attributes -------------
    hq_sum_x = 0  # composite score 
    hq_tot_x = 0
    for habitat_attribute_x in life_stage_habitat_attributes:

        #print('-------------------------------habitat_attribute_x------------------: ' + habitat_attribute_x)
        
        # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
        if any(habitat_quality_scores_row.columns == habitat_attribute_x):
            #print('habitat_quality_scores_row[habitat_attribute_x]: ' )
            #print(habitat_quality_scores_row[habitat_attribute_x])
            # --------- convert habitat quality score to float -------
            habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])

            # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
            hq_tot_x = hq_tot_x + 1

            # ---------if the habitat quality score is numeric (non NaN) --------
            if math.isnan(habitat_score_x) == False:
                #print('------in math loop-----------')
                #print('habitat_score_x: ')
                #print(habitat_score_x)
                #print('hq_sum_x: ')
                #print(hq_sum_x)
                hq_sum_x = hq_sum_x + habitat_score_x


            # ------------ sum of habitat quality scores ----------
            #print('habitat_score_x: ')
            #print(habitat_score_x)
            #print('hq_sum_x: ')
            #print(hq_sum_x)
            summer_rearing_sum = hq_sum_x
            # ------------ maximum potential score ----------
            summer_rearing_max = (hq_tot_x * 5)
            # -------- percent functional of habitat for this life stage ----------
            summer_rearing_prcnt = hq_sum_x / summer_rearing_max

        else:
            summer_rearing_sum = 'NA'
            summer_rearing_max = 'NA'
            summer_rearing_prcnt = 'NA'

else:
    summer_rearing_sum = 'NA'
    summer_rearing_max = 'NA'
    summer_rearing_prcnt = 'NA'

# --------------------------------------------------------------------------------------------
#                    Spring Chinook - Winter Rearing
# --------------------------------------------------------------------------------------------

if any(reach_life_stage_presence_Spring_Chinook_row.SPCH_Winter_Rearing == 1):

    life_stage_habitat_attributes = habitat_attribute_lifestage_Spring_Chinook.loc[habitat_attribute_lifestage_Spring_Chinook['Life Stage'] == 'Winter Rearing']
    life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']

    # -------------- loop through life stage attributes -------------
    hq_sum_x = 0  # composite score 
    hq_tot_x = 0
    for habitat_attribute_x in life_stage_habitat_attributes:

        # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
        if any(habitat_quality_scores_row.columns == habitat_attribute_x):

            # --------- convert habitat quality score to float -------
            habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])

            # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
            hq_tot_x = hq_tot_x + 1

            # ---------if the habitat quality score is numeric (non NaN) --------
            if math.isnan(habitat_score_x) == False:
                hq_sum_x = hq_sum_x + habitat_score_x

            # ------------ sum of habitat quality scores ----------
            winter_rearing_sum = hq_sum_x
            # ------------ maximum potential score ----------
            winter_rearing_max = (hq_tot_x * 5)
            # -------- percent functional of habitat for this life stage ----------
            winter_rearing_prcnt = hq_sum_x / winter_rearing_max

    else:
        winter_rearing_sum = 'NA'
        winter_rearing_max = 'NA'
        winter_rearing_prcnt = 'NA'

else:
    winter_rearing_sum = 'NA'
    winter_rearing_max = 'NA'
    winter_rearing_prcnt = 'NA'

# --------------------------------------------------------------------------------------------
#                    Spring Chinook - Smolt Emigration
# --------------------------------------------------------------------------------------------


if any(reach_life_stage_presence_Spring_Chinook_row.SPCH_Smolt_Emigration == 1):

    life_stage_habitat_attributes = habitat_attribute_lifestage_Spring_Chinook.loc[habitat_attribute_lifestage_Spring_Chinook['Life Stage'] == 'Smolt Outmigration']
    life_stage_habitat_attributes = life_stage_habitat_attributes['Habitat Attribute']

    # -------------- loop through life stage attributes -------------
    hq_sum_x = 0  # composite score 
    hq_tot_x = 0
    for habitat_attribute_x in life_stage_habitat_attributes:

        # -------------- if habitat attribute linked to life stage is in habitat quality scores ----------
        if any(habitat_quality_scores_row.columns == habitat_attribute_x):

            # --------- convert habitat quality score to float -------
            habitat_score_x = float(habitat_quality_scores_row[habitat_attribute_x])

            # ------------- count number of life stage habitat attributes that are in habitat quality scores -----
            hq_tot_x = hq_tot_x + 1

            # ---------if the habitat quality score is numeric (non NaN) --------
            if math.isnan(habitat_score_x) == False:
                hq_sum_x = hq_sum_x + habitat_score_x

            # ------------ sum of habitat quality scores ----------
            smolt_emigration_sum = hq_sum_x
            # ------------ maximum potential score ----------
            smolt_emigration_max = (hq_tot_x * 5)
            # -------- percent functional of habitat for this life stage ----------
            smolt_emigration_prcnt = hq_sum_x / smolt_emigration_max

        #else:
            #smolt_emigration_sum = 'NA'
            #smolt_emigration_max = 'NA'
            #smolt_emigration_prcnt = 'NA'
else:
    smolt_emigration_sum = 'NA'
    smolt_emigration_max = 'NA'
    smolt_emigration_prcnt = 'NA'




HOLDING OUTPUT ------------:
NA
NA
NA
-------------------------------habitat_attribute_x------------------: % Fines/Embeddedness
-------------------------------habitat_attribute_x------------------: Contaminants
-------------------------------habitat_attribute_x------------------: Flow- Scour
-------------------------------habitat_attribute_x------------------: Flow- Summer Base Flow
habitat_quality_scores_row[habitat_attribute_x]: 
67    5.0
Name: Flow- Summer Base Flow, dtype: float64
------in math loop-----------
habitat_score_x: 
5.0
hq_sum_x: 
0
habitat_score_x: 
5.0
hq_sum_x: 
5.0
-------------------------------habitat_attribute_x------------------: Harassment
-------------------------------habitat_attribute_x------------------: Hybridization
-------------------------------habitat_attribute_x------------------: Icing
-------------------------------habitat_attribute_x------------------: Off-Channel- Side-Channels
habitat_quality_scores_row[habitat_attribute_x]: 
67   NaN
Name: Off

In [575]:
# --------------------------------------------------------------------------------------------
#            Add Life Stage Presence/Absence data to row
# --------------------------------------------------------------------------------------------
life_stage_presence = reach_life_stage_presence_Spring_Chinook_row[['SPCH_Adult_Migration','SPCH_Holding','SPCH_Spawning',
                'SPCH_Fry_Colonization','SPCH_Summer_Rearing','SPCH_Winter_Rearing','SPCH_Smolt_Emigration']]
life_stage_presence = life_stage_presence.set_index(habitat_quality_scores_row.index)

habitat_quality_scores_row_Spring_Chinook = habitat_quality_scores_row.join(  life_stage_presence )



In [576]:
# habitat_quality_scores_row_Spring_Chinook

In [577]:
# --------------------------------------------------------------------------------------------
#            Combine all the Spring Chinook
# --------------------------------------------------------------------------------------------

#habitat_quality_row_x2 = pd.DataFrame([adult_migration_sum, adult_migration_prcnt])
habitat_quality_scores_row_Spring_Chinook = habitat_quality_scores_row_Spring_Chinook.join(pd.DataFrame(
{
    'adult_migration_sum': adult_migration_sum,
    'adult_migration_max': adult_migration_max,
    'adult_migration_prcnt': adult_migration_prcnt,
    'holding_sum': holding_sum,
    'holding_max': holding_max,
    'holding_prcnt': holding_prcnt,
    'spawning_sum': spawning_sum,
    'spawning_max': spawning_max,
    'spawning_prcnt': spawning_prcnt,
    'fry_sum': fry_sum,
    'fry_max': fry_max,
    'fry_prcnt': fry_prcnt,
    'summer_rearing_sum': summer_rearing_sum,
    'summer_rearing_max': summer_rearing_max,
    'summer_rearing_prcnt': summer_rearing_prcnt,
    'winter_rearing_sum': winter_rearing_sum,
    'winter_rearing_max': winter_rearing_max,
    'winter_rearing_prcnt': winter_rearing_prcnt,
    'smolt_emigration_sum': smolt_emigration_sum,
    'smolt_emigration_max': smolt_emigration_max,
    'smolt_emigration_prcnt': smolt_emigration_prcnt,

}, index=habitat_quality_scores_row_Spring_Chinook.index ), how='right' )

In [578]:
# habitat_quality_scores_row_Spring_Chinook

In [579]:
# --------------------------------------------------------------------------------------------
#            Add Life Stage Presence/Absence data to row
# --------------------------------------------------------------------------------------------
life_stage_presence = reach_life_stage_presence_Spring_Chinook_row[['SPCH_Adult_Migration','SPCH_Holding','SPCH_Spawning',
                'SPCH_Fry_Colonization','SPCH_Summer_Rearing','SPCH_Winter_Rearing','SPCH_Smolt_Emigration']]
life_stage_presence = life_stage_presence.set_index(habitat_quality_scores_row.index)

habitat_quality_scores_row.join(  life_stage_presence )

,ReachName,Assessment Unit,Subbasin,Spring.Chinook.Reach,Steelhead.Reach,AU Rank Sp CHN,AU Rank STL,Confinement,Number of Life Stages- SPCHN,Number of Life Stages- STL,Temperature- Rearing,Flow- Summer Base Flow,Riparian- Structure,Riparian-Disturbance,Riparian- Canopy Cover,Riparian,Coarse Substrate,Cover- Wood,Pool Quantity& Quality,Off-Channel- Floodplain,Off-Channel- Side-Channels,Channel Stability,Bank Stability,Stability,SPCH_Adult_Migration,SPCH_Holding,SPCH_Spawning,SPCH_Fry_Colonization,SPCH_Summer_Rearing,SPCH_Winter_Rearing,SPCH_Smolt_Emigration
67,Alder Creek 01,Lower Chiwawa River,Wenatchee,yes,yes,1,2,5,3,6,1.0,5.0,NaN,NaN,NaN,NaN,5.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,0,0,1,1,1,0,0


In [580]:

# --------------------------------------------------------------------------------------------
#            Add Life Stage Presence/Absence data to row
# --------------------------------------------------------------------------------------------

# habitat_quality_scores_row_Spring_Chinook =     
habitat_quality_scores_row.join( 
reach_life_stage_presence_Spring_Chinook_row[['SPCH_Adult_Migration','SPCH_Holding','SPCH_Spawning',
                'SPCH_Fry_Colonization','SPCH_Summer_Rearing','SPCH_Winter_Rearing','SPCH_Smolt_Emigration']],
    how = 'left'
 )

,ReachName,Assessment Unit,Subbasin,Spring.Chinook.Reach,Steelhead.Reach,AU Rank Sp CHN,AU Rank STL,Confinement,Number of Life Stages- SPCHN,Number of Life Stages- STL,Temperature- Rearing,Flow- Summer Base Flow,Riparian- Structure,Riparian-Disturbance,Riparian- Canopy Cover,Riparian,Coarse Substrate,Cover- Wood,Pool Quantity& Quality,Off-Channel- Floodplain,Off-Channel- Side-Channels,Channel Stability,Bank Stability,Stability,SPCH_Adult_Migration,SPCH_Holding,SPCH_Spawning,SPCH_Fry_Colonization,SPCH_Summer_Rearing,SPCH_Winter_Rearing,SPCH_Smolt_Emigration
67,Alder Creek 01,Lower Chiwawa River,Wenatchee,yes,yes,1,2,5,3,6,1.0,5.0,NaN,NaN,NaN,NaN,5.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## ---------------------------------------------------------------------------------------------------------------------------

# -
# Step 4: Develop Single Table for Potential Limiting Factors by Assessment Unit (Tier 1 watersheds only with Life Stage "High Priority")
# -

## ---------------------------------------------------------------------------------------------------------------------------

In [581]:
# NOTE: note sure if I used this Function
def Reaches_in_priority_AUs(AU_Life_Stage_Priority_Criteria_AUs):
    
    Reach_Life_Stage_Presence_Criteria = reach_life_stage_presence.loc[reach_life_stage_presence[Reach_Life_Stage_Presence_vars[variable_name][life_stage_val]].isin(Reach_Life_Stage_Presence)]
    Reach_Life_Stage_Presence_Criteria_ReachName = Reach_Life_Stage_Presence_Criteria.ReachName
    Species_data_frame_Life_Stage = Species_data_frame_Life_Stage.loc[Species_data_frame_Life_Stage['ReachName'].isin(Reach_Life_Stage_Presence_Criteria_ReachName)] 
    print('number of reaches/row after Life Stage Presence criteria: ' + str(Species_data_frame_Life_Stage.shape[0]) )

In [582]:
# (DONE) Step 1: Create a loop to loop through the life stages
# (DONE) Step 2: identify the priority AUs
# (DONE) Step 3: count the total number of reaches in the priority AUs
# (DONE) Step 4: identify the habitat attributes for each reach
# (DONE) Step 5: count the reaches for EACH habitat attribute with that limiting factor
# (DONE) Step 6: count percent of reaches with that limiting factor
# Step 7: generate the input LF score
# Step 8: add "professoinal judgement" column - need to clarify with Greer how this is developed

## ---------------------------------------------------------------------------------------------------------------------------
## For loop to generate Limiting Factor Reach Percentage Tables
## ---------------------------------------------------------------------------------------------------------------------------

In [583]:
output_path_spring_chinook = output_location + 'AU_scale_Step2_Limiting_Factors_Score_Spring_Chinook.xlsx'
output_path_steelhead = output_location + 'AU_scale_Step2_Limiting_Factors_Score_Steelhead.xlsx'

## ------------------------------------- Function to output Priority AUs ---------------------------

In [584]:
# Function to generate Priority AUs based on a) life stage and b) desired priority of life stage

def Priority_AUs_for_Life_Stage_FUNCTION(species, life_stage_val, Life_Stage_Priority):
    
    if species == "Spring Chinook":

        # ------- establish species variable name -----
        variable_name = 'variable_SPCH'

    elif species == "Steelhead":

        # ------- establish species variable name -----
        variable_name = 'variable_SH'

    else:

        
        print('Incorrectly entered species name - re-type species name')
        return
        

    # -------------------------- pull column for the life stage ----------
    AU_Life_Stage_Priority_Criteria = AU_life_stage_priorities.loc[AU_life_stage_priorities[Life_Stage_Priority_vars[variable_name][life_stage_val]].isin(Life_Stage_Priority)]

    # --------------------- Return List of AU's with the Life Stage Priority ---------
    AU_Life_Stage_Priority_Criteria_NAMES_ONLY = AU_Life_Stage_Priority_Criteria['AU']
    return(AU_Life_Stage_Priority_Criteria_NAMES_ONLY)
    #print(AU_Life_Stage_Priority_Criteria)
    
    

## ------------------------------------- Spring Chinook ---------------------------

In [585]:
species = "Spring Chinook"
Limiting_Factor_Prcnt_Reaches_DataFrame = pd.DataFrame([])
for i in range(index_vals.shape[0]):
    life_stage_name = Life_Stage_Priority_vars.variable[i]
    print('starting output for Life Stage: ' + str(life_stage_name) )  #print('starting output for Species: ' + species + "for the Life Stage: " + life_stage)
    
    # ------------------------------------------------------------------------------
    #       Establish species name variable
    # ------------------------------------------------------------------------------

    if species == "Spring Chinook":

        # ------- establish species variable name -----
        variable_name = 'variable_SPCH'
        # -------------- species data frame --------
        Species_data_frame = Spring_Chinook_data_frame
        # ------------- habitat attribute data frame -----------
        Species_Life_Stage_Habitat_Attributes = Spring_Chinook_Life_Stage_Habitat_Attributes
        # ----------- Use Spring ChinooSpecies_data_frame_Life_Stage.Subbasink Reaches-Life Stage data --------
        reach_life_stage_presence = reach_life_stage_presence_Spring_Chinook
        # ---------------- Output correct column names -------------
        output_column_AU_column = 'AU Tier Spring CHN' 

    elif species == "Steelhead":

        # ------- establish species variable name -----
        variable_name = 'variable_SH'
        # -------------- species data frame --------
        Species_data_frame = Steelhead_data_frame
        # ------------- habitat attribute data frame -----------
        Species_Life_Stage_Habitat_Attributes = Steelhead_Life_Stage_Habitat_Attributes
        # ----------- Use Steelhead Reaches-Life Stage data --------
        reach_life_stage_presence = reach_life_stage_presence_Steelhead
        # ---------------- Output correct column names -------------
        output_column_AU_column = 'AU Tier Steelhead' 

    else:
        
        print('Incorrectly entered species name - re-type species name')

        
    # -------------- test variables ---------- 
    life_stage = life_stages_all.life_stages[i]
    
    # --------------------------------- life stage index variable --------------------------------      
    life_stage_val = int(index_vals[life_stages_all.life_stages.isin([life_stage])])
    life_stage_val_list = [life_stage_val]
    
    # --------- generate priority AUs --------
    AU_Life_Stage_Priority_Criteria_AUs = Priority_AUs_for_Life_Stage_FUNCTION(species, life_stage_val, Life_Stage_Priority)
    
    # ---------------------- reaches in priority AUs ---------------
    Reaches_priority_AUs = reach_life_stage_presence[reach_life_stage_presence.AssessmentUnit.isin(AU_Life_Stage_Priority_Criteria_AUs)]

    # -------------------- generate number of reaches in each priority AU ---------------------
    for AU_x in Reaches_priority_AUs.AssessmentUnit.unique():
        
     
        # ------------ upload all reaches -------
        all_AU_reaches = Reaches_priority_AUs[Reaches_priority_AUs.AssessmentUnit.isin([AU_x])]
        
        # --------------- total number of reaches ----------
        total_reaches_x = all_AU_reaches.shape[0]
        
        # ----------------- list of Habitat Attributes for this specific Life Stage ---------------
        Life_Stage_Habitat_Criteria = Species_Life_Stage_Habitat_Attributes.loc[Species_Life_Stage_Habitat_Attributes['Life Stage'].isin( [Life_Stage_Habitat_Attributes_vars['variable'][life_stage_val] ])]   
        habitat_attributes_list = Life_Stage_Habitat_Criteria['Habitat Attribute'].values.tolist()
        
        # -------------------- data frame ----------------
        Species_data_frame_Life_Stage = Species_data_frame.loc[Species_data_frame['Assessment Unit'].isin([AU_x])] 

        # --------------- Reaches and Habitat Attributes for this Life Stage ----
        Life_Stage_Habitat_Criteria_2 = Species_data_frame_Life_Stage.loc[Species_data_frame_Life_Stage['Habitat Attribute'].isin(habitat_attributes_list)]

        # ----------- start list for each Assessment Unit (AU) ---------
        AU_list = pd.DataFrame([])
        
        # ---------------- Loop through each habitat attribute and count the reaches with limiting factors ---------
        for habitat_attribute_x in habitat_attributes_list:
            
            # -------------------- reaches in the AU ---------------
            reaches_in_AU = Life_Stage_Habitat_Criteria_2[Life_Stage_Habitat_Criteria_2['Habitat Attribute'].isin([habitat_attribute_x])]
            # ----------------------- number of reaches in the AU ---------------------
            number_of_reaches_x = reaches_in_AU.shape[0]
            # --------------- percent of reaches with this habitat attribute as limiting factor ------------
            percent_of_reaches_x = number_of_reaches_x / total_reaches_x
            
            # --------------------------------------------------------------------------------------
            #   append reach data to data frame --
            # --------------------------------------------------------------------------------------
            
            subbasin_x = all_AU_reaches.Basin.iloc[0] # sub-basin name
            
            # -------- generate scoring for limiting factor percent of reaches -----
            
            #----- percent below first cut off (lowest score) ---
            if any(percent_of_reaches_x <= score_cutoff_percent_of_reaches.iloc[0]):
                limiting_factor_score_x = scores_limiting_factors_prcnt_reaches[0]
            
            # ------ percent above second cut off (highest score) -----
            elif any(percent_of_reaches_x > score_cutoff_percent_of_reaches.iloc[1]):
                limiting_factor_score_x = scores_limiting_factors_prcnt_reaches[2]
                
            # ------- percent between both cut offs (middle score) ------
            else:
                limiting_factor_score_x = scores_limiting_factors_prcnt_reaches[1]
                    
            # ------ percent of reaches -------
            percent_of_reaches_x = percent_of_reaches_x * 100 # convert to percentage
            percent_of_reaches_x = round(percent_of_reaches_x, 1)
            

            # ---------- combine all
            array_x = pd.DataFrame([species, life_stage, subbasin_x, AU_x, total_reaches_x,  habitat_attribute_x, 
                                    number_of_reaches_x, percent_of_reaches_x, limiting_factor_score_x])
            array_x = array_x.transpose()
            AU_list = AU_list.append(array_x)
            
          
        Limiting_Factor_Prcnt_Reaches_DataFrame = Limiting_Factor_Prcnt_Reaches_DataFrame.append(AU_list)  
    print(' ----------------------------------- Limiting_Factor_Prcnt_Reaches_DataFrame --------------------------------')
    print(Limiting_Factor_Prcnt_Reaches_DataFrame.head())
# ------------------ Write out the combined data ----------------------           
Limiting_Factor_Prcnt_Reaches_DataFrame.columns = ['Species','Life Stage','Basin', 'Assessment Unit',
            'Total Reaches in AU','Habitat Attribute', 'Count of Reaches with LF','Percent of Reaches', 'Limiting Factor Score']
Limiting_Factor_Prcnt_Reaches_DataFrame.to_excel(output_path_spring_chinook, index = False)

starting output for Life Stage: Adult Migration
 ----------------------------------- Limiting_Factor_Prcnt_Reaches_DataFrame --------------------------------
Empty DataFrame
Columns: []
Index: []
starting output for Life Stage: Holding and Maturation
 ----------------------------------- Limiting_Factor_Prcnt_Reaches_DataFrame --------------------------------
                0                       1          2  \
0  Spring Chinook  Holding and Maturation  Wenatchee   
0  Spring Chinook  Holding and Maturation  Wenatchee   
0  Spring Chinook  Holding and Maturation  Wenatchee   
0  Spring Chinook  Holding and Maturation  Wenatchee   
0  Spring Chinook  Holding and Maturation  Wenatchee   

                              3   4                       5  6     7  8  
0  Wenatchee River-Beaver Creek  18         Cover- Boulders  0     0  1  
0  Wenatchee River-Beaver Creek  18   Cover- Undercut Banks  6  33.3  3  
0  Wenatchee River-Beaver Creek  18             Cover- Wood  9    50  3  
0  Wen

## ------------------------------------- Steelhead ---------------------------

In [586]:
species = "Steelhead"
Limiting_Factor_Prcnt_Reaches_DataFrame = pd.DataFrame([])
for i in range(index_vals.shape[0]):
    life_stage_name = Life_Stage_Priority_vars.variable[i]
    print('starting output for Life Stage: ' + str(life_stage_name) )  #print('starting output for Species: ' + species + "for the Life Stage: " + life_stage)
    
    # ------------------------------------------------------------------------------
    #       Establish species name variable
    # ------------------------------------------------------------------------------

    if species == "Spring Chinook":

        # ------- establish species variable name -----
        variable_name = 'variable_SPCH'
        # -------------- species data frame --------
        Species_data_frame = Spring_Chinook_data_frame
        # ------------- habitat attribute data frame -----------
        Species_Life_Stage_Habitat_Attributes = Spring_Chinook_Life_Stage_Habitat_Attributes
        # ----------- Use Spring ChinooSpecies_data_frame_Life_Stage.Subbasink Reaches-Life Stage data --------
        reach_life_stage_presence = reach_life_stage_presence_Spring_Chinook
        # ---------------- Output correct column names -------------
        output_column_AU_column = 'AU Tier Spring CHN' 

    elif species == "Steelhead":

        # ------- establish species variable name -----
        variable_name = 'variable_SH'
        # -------------- species data frame --------
        Species_data_frame = Steelhead_data_frame
        # ------------- habitat attribute data frame -----------
        Species_Life_Stage_Habitat_Attributes = Steelhead_Life_Stage_Habitat_Attributes
        # ----------- Use Steelhead Reaches-Life Stage data --------
        reach_life_stage_presence = reach_life_stage_presence_Steelhead
        # ---------------- Output correct column names -------------
        output_column_AU_column = 'AU Tier Steelhead' 

    else:
        
        print('Incorrectly entered species name - re-type species name')

        
    # -------------- test variables ---------- 
    life_stage = life_stages_all.life_stages[i]
    
    # --------------------------------- life stage index variable --------------------------------      
    life_stage_val = int(index_vals[life_stages_all.life_stages.isin([life_stage])])
    life_stage_val_list = [life_stage_val]
    
    # --------- generate priority AUs --------
    AU_Life_Stage_Priority_Criteria_AUs = Priority_AUs_for_Life_Stage_FUNCTION(species, life_stage_val, Life_Stage_Priority)
    
    # ---------------------- reaches in priority AUs ---------------
    Reaches_priority_AUs = reach_life_stage_presence[reach_life_stage_presence.AssessmentUnit.isin(AU_Life_Stage_Priority_Criteria_AUs)]

    # -------------------- generate number of reaches in each priority AU ---------------------
    for AU_x in Reaches_priority_AUs.AssessmentUnit.unique():
        
     
        # ------------ upload all reaches -------
        all_AU_reaches = Reaches_priority_AUs[Reaches_priority_AUs.AssessmentUnit.isin([AU_x])]
        
        # --------------- total number of reaches ----------
        total_reaches_x = all_AU_reaches.shape[0]
        
        # ----------------- list of Habitat Attributes for this specific Life Stage ---------------
        Life_Stage_Habitat_Criteria = Species_Life_Stage_Habitat_Attributes.loc[Species_Life_Stage_Habitat_Attributes['Life Stage'].isin( [Life_Stage_Habitat_Attributes_vars['variable'][life_stage_val] ])]   
        habitat_attributes_list = Life_Stage_Habitat_Criteria['Habitat Attribute'].values.tolist()
        
        # -------------------- data frame ----------------
        Species_data_frame_Life_Stage = Species_data_frame.loc[Species_data_frame['Assessment Unit'].isin([AU_x])] 

        # --------------- Reaches and Habitat Attributes for this Life Stage ----
        Life_Stage_Habitat_Criteria_2 = Species_data_frame_Life_Stage.loc[Species_data_frame_Life_Stage['Habitat Attribute'].isin(habitat_attributes_list)]

        # ----------- start list for each Assessment Unit (AU) ---------
        AU_list = pd.DataFrame([])
        
        # ---------------- Loop through each habitat attribute and count the reaches with limiting factors ---------
        for habitat_attribute_x in habitat_attributes_list:
            
            # -------------------- reaches in the AU ---------------
            reaches_in_AU = Life_Stage_Habitat_Criteria_2[Life_Stage_Habitat_Criteria_2['Habitat Attribute'].isin([habitat_attribute_x])]
            # ----------------------- number of reaches in the AU ---------------------
            number_of_reaches_x = reaches_in_AU.shape[0]
            # --------------- percent of reaches with this habitat attribute as limiting factor ------------
            percent_of_reaches_x = number_of_reaches_x / total_reaches_x
            
            # --------------------------------------------------------------------------------------
            #   append reach data to data frame --
            # --------------------------------------------------------------------------------------
            
            subbasin_x = all_AU_reaches.Basin.iloc[0] # sub-basin name
            
            # -------- generate scoring for limiting factor percent of reaches -----
            
            #----- percent below first cut off (lowest score) ---
            if any(percent_of_reaches_x <= score_cutoff_percent_of_reaches.iloc[0]):
                limiting_factor_score_x = scores_limiting_factors_prcnt_reaches[0]
            
            # ------ percent above second cut off (highest score) -----
            elif any(percent_of_reaches_x > score_cutoff_percent_of_reaches.iloc[1]):
                limiting_factor_score_x = scores_limiting_factors_prcnt_reaches[2]
                
            # ------- percent between both cut offs (middle score) ------
            else:
                limiting_factor_score_x = scores_limiting_factors_prcnt_reaches[1]
                    
            # ------ percent of reaches -------
            percent_of_reaches_x = percent_of_reaches_x * 100 # convert to percentage
            percent_of_reaches_x = round(percent_of_reaches_x, 1)
            

            # ---------- combine all
            array_x = pd.DataFrame([species, life_stage, subbasin_x, AU_x, total_reaches_x,  habitat_attribute_x, 
                                    number_of_reaches_x, percent_of_reaches_x, limiting_factor_score_x])
            array_x = array_x.transpose()
            AU_list = AU_list.append(array_x)
          
        Limiting_Factor_Prcnt_Reaches_DataFrame = Limiting_Factor_Prcnt_Reaches_DataFrame.append(AU_list)
    print(' ----------------------------------- Limiting_Factor_Prcnt_Reaches_DataFrame --------------------------------')
    print(Limiting_Factor_Prcnt_Reaches_DataFrame.head())
# ------------------ Write out the combined data ----------------------           
Limiting_Factor_Prcnt_Reaches_DataFrame.columns = ['Species','Life Stage','Basin', 'Assessment Unit',
            'Total Reaches in AU','Habitat Attribute', 'Count of Reaches with LF','Percent of Reaches', 'Limiting Factor Score']
Limiting_Factor_Prcnt_Reaches_DataFrame.to_excel(output_path_steelhead, index = False)

starting output for Life Stage: Adult Migration
 ----------------------------------- Limiting_Factor_Prcnt_Reaches_DataFrame --------------------------------
           0                1          2                      3   4  \
0  Steelhead  Adult Migration   Okanogan    Lower Antoine Creek   6   
0  Steelhead  Adult Migration  Wenatchee  Upper Peshastin Creek  17   
0  Steelhead  Adult Migration  Wenatchee     Lower Icicle Creek   8   
0  Steelhead  Adult Migration   Okanogan          Johnson Creek   8   
0  Steelhead  Adult Migration   Okanogan  Lower Loup Loup Creek   4   

                        5  6     7  8  
0  Flow- Summer Base Flow  0     0  1  
0  Flow- Summer Base Flow  0     0  1  
0  Flow- Summer Base Flow  3  37.5  3  
0  Flow- Summer Base Flow  1  12.5  1  
0  Flow- Summer Base Flow  1    25  1  
starting output for Life Stage: Holding and Maturation
 ----------------------------------- Limiting_Factor_Prcnt_Reaches_DataFrame --------------------------------
          

In [587]:
# output_path_steelhead

## ---------------------------------------------------------------------------------------------------------------------------

# End of code - below is code to play with data


## ---------------------------------------------------------------------------------------------------------------------------

In [ ]:
Reach_Life_Stage_Presence_Criteria = reach_life_stage_presence.loc[reach_life_stage_presence[Reach_Life_Stage_Presence_vars[variable_name][life_stage_val]].isin(Reach_Life_Stage_Presence)]


In [ ]:
Life_Stage_Priority_vars

In [ ]:
Reach_Life_Stage_Presence_vars[variable_name][life_stage_val]

In [ ]:
Life_Stage_Priority_vars[variable_name][life_stage_val]

In [ ]:

# -------------- test variables ----------  
i = 4
life_stage = life_stages_all.life_stages[i]
species = "Spring Chinook"
output_path = Output_Spring_Chinook_list[i]

# ------------------------------------------------------------------------------
#      Initiate Data Processing
# ------------------------------------------------------------------------------

print('starting output for Species: ' + species + "for the Life Stage: " + life_stage)

# ------------------------------------------------------------------------------
#       Establish species name variable
# ------------------------------------------------------------------------------

# ------------------------------------------------------------------------------
#      Initiate Data Processing
# ------------------------------------------------------------------------------

print('starting output for Species: ' + species + "for the Life Stage: " + life_stage)

# ------------------------------------------------------------------------------
#       Establish species name variable
# ------------------------------------------------------------------------------

if species == "Spring Chinook":

    # ------- establish species variable name -----
    variable_name = 'variable_SPCH'
    # -------------- species data frame --------
    Species_data_frame = Spring_Chinook_data_frame
    # ------------- habitat attribute data frame -----------
    Species_Life_Stage_Habitat_Attributes = Spring_Chinook_Life_Stage_Habitat_Attributes
    # ----------- Use Spring Chinook Reaches-Life Stage data --------
    reach_life_stage_presence = reach_life_stage_presence_Spring_Chinook
    # ---------------- Output correct column names -------------
    output_column_AU_column = 'AU Tier Spring CHN' 

elif species == "Steelhead":

    # ------- establish species variable name -----
    variable_name = 'variable_SH'
    # -------------- species data frame --------
    Species_data_frame = Steelhead_data_frame
    # ------------- habitat attribute data frame -----------
    Species_Life_Stage_Habitat_Attributes = Steelhead_Life_Stage_Habitat_Attributes
    # ----------- Use Steelhead Reaches-Life Stage data --------
    reach_life_stage_presence = reach_life_stage_presence_Steelhead
    # ---------------- Output correct column names -------------
    output_column_AU_column = 'AU Tier Steelhead' 

else:

    print('Incorrectly entered species name - re-type species name')
   # return


# --------------------------------- life stage index variable --------------------------------      
life_stage_val = int(index_vals[life_stages_all.life_stages.isin([life_stage])])
life_stage_val_list = [life_stage_val]

# ------------------------------------------------------------------------------
#       Establish index of life stage (always in same order)
# ------------------------------------------------------------------------------

#life_stage_val

# ------------------------------------------------------------------------------
#       AU Life Stage Priority for this life stage
# ------------------------------------------------------------------------------


AU_Life_Stage_Priority_Criteria = AU_life_stage_priorities.loc[AU_life_stage_priorities[Life_Stage_Priority_vars[variable_name][life_stage_val]].isin(Life_Stage_Priority)]
AU_Life_Stage_Priority_Criteria_AUs = AU_Life_Stage_Priority_Criteria.AU
Species_data_frame_Life_Stage = Species_data_frame.loc[Species_data_frame['Assessment Unit'].isin(AU_Life_Stage_Priority_Criteria_AUs)] 
print('number of reaches/row after AU Life Stage Priority criteria: ' + str(Species_data_frame_Life_Stage.shape[0]) )

# ------------------------------------------------------------------------------
#       Pull reaches with this Life Stage presence
# ------------------------------------------------------------------------------

Reach_Life_Stage_Presence_Criteria = reach_life_stage_presence.loc[reach_life_stage_presence[Reach_Life_Stage_Presence_vars[variable_name][life_stage_val]].isin(Reach_Life_Stage_Presence)]
Reach_Life_Stage_Presence_Criteria_ReachName = Reach_Life_Stage_Presence_Criteria.ReachName
Species_data_frame_Life_Stage = Species_data_frame_Life_Stage.loc[Species_data_frame_Life_Stage['ReachName'].isin(Reach_Life_Stage_Presence_Criteria_ReachName)] 
print('number of reaches/row after Life Stage Presence criteria: ' + str(Species_data_frame_Life_Stage.shape[0]) )

# ------------------------------------------------------------------------------
#       Identify the Habitat Attributes for the Specific Life Stage
# ------------------------------------------------------------------------------
Life_Stage_Habitat_Criteria = life_stage_habitat.loc[life_stage_habitat['Life Stage'].isin([Life_Stage_Habitat_Attributes_vars['variable'][life_stage_val] ])]
Reach_Life_Stage_Presence_Criteria_ReachName = Reach_Life_Stage_Presence_Criteria.ReachName
Species_data_frame_Life_Stage = Species_data_frame_Life_Stage.loc[Species_data_frame_Life_Stage['ReachName'].isin(Reach_Life_Stage_Presence_Criteria_ReachName)] 
print('number of reaches/row after Life Stage Presence criteria: ' + str(Species_data_frame_Life_Stage.shape[0]) )



# ------------------ Habitat Attributes for this Life Stage --------------------------------
Life_Stage_Habitat_Criteria = Species_Life_Stage_Habitat_Attributes.loc[Species_Life_Stage_Habitat_Attributes['Life Stage'].isin(Life_Stage_Habitat_Attributes_vars['variable'][life_stage_val_list])]   

# --------------- List of Reaches and Habitat Attributes based on this Life Stage --------------

# ----------------- list of Habitat Attributes for this specific Life Stage ---------------
habitat_attributes_list = Life_Stage_Habitat_Criteria['Habitat Attribute'].values.tolist()
# --------------- Reaches and Habitat Attributes for this Life Stage
Life_Stage_Habitat_Criteria_2 = Species_data_frame_Life_Stage.loc[Species_data_frame_Life_Stage['Habitat Attribute'].isin(habitat_attributes_list)]


# ------------------------------------------------------------------------------
#      Prepare all the Variables for the Output Table
# ------------------------------------------------------------------------------

# Example Output Variables (for Spring Chinook - Holding and Maturation):  subbasin,  AU, AU Tier SprChn, 
#           Spch_Holding_priority (high priority),  ReachName,  SPCH_Holding,  Species, Life_Stage, Habitat_Attribute,  
#          DRAFT Habitat Attribute Score

# Update Assessment Unit column name to be identical to Life Stage data frame
AU_Life_Stage_Priority_Criteria = AU_Life_Stage_Priority_Criteria.rename(columns = {'AU':'Assessment Unit'})

# Append Life Stage Habitat Criteria and AU life Stage Priority data frame to include AU Life Stage Priority
Life_Stage_Habitat_Criteria_combined = pd.merge(Life_Stage_Habitat_Criteria_2, AU_Life_Stage_Priority_Criteria, how='left',  on = ['Assessment Unit'])

# Append Life Stage Habitat Criteria and AU life Stage Priority data frame to include AU Life Stage Priority
Life_Stage_Habitat_Criteria_combined = pd.merge(Life_Stage_Habitat_Criteria_combined, Reach_Life_Stage_Presence_Criteria, how='left',  on = ['ReachName'])

# generate column of species name
rows_x = Life_Stage_Habitat_Criteria_combined.shape[0]  # get number of rows in Life_Stage_Habitat_Criteria_combined
col_x = pd.DataFrame({'Species': [species] * rows_x})  # create columns
Life_Stage_Habitat_Criteria_combined['Species'] = col_x          # add column

# add Life Stage name
col_x = pd.DataFrame({'Species': [Life_Stage_Priority_vars['variable'][life_stage_val]] * rows_x})  # create columns
Life_Stage_Habitat_Criteria_combined['Life_Stage'] = col_x          # add column


# ------------------------------------------------------------------------------
#      Create Output Table
# ------------------------------------------------------------------------------

# create output that matches the output table in Access
Output_data_frame = Life_Stage_Habitat_Criteria_combined[['Subbasin_x', 'Assessment Unit', output_column_AU_column,  
       Life_Stage_Priority_vars[variable_name][life_stage_val],  'ReachName',  Reach_Life_Stage_Presence_vars[variable_name][life_stage_val],
      'Species', 'Life_Stage' , 'Habitat Attribute', 'DRAFT Habitat Attribute Score (Currently MAX Score OR "0" if NA)' ]]
Output_data_frame = Output_data_frame.rename(columns={'Subbasin_x':'Subbasin', 
                                  'DRAFT Habitat Attribute Score (Currently MAX Score OR "0" if NA)' : 'Habitat Attribute Score'})
# re-order based on ReachName
Output_data_frame_X = Output_data_frame.sort_values(by=['ReachName'])
# Output_data_frame_X.iloc[0:78]    # just can use to output data


# ------------------------------------------------------------------------------
#       Generate Columns for Output
# ------------------------------------------------------------------------------

Output_data_frame_X.to_excel(output_path, index = False)

print('completed output for Species: ' + species + "for the Life Stage: " + life_stage)
print(' ----------------------------------------------------------------------------------')
print("      ")

In [ ]:
life_stage_val = 3
Life_Stage_Priority_vars[variable_name][life_stage_val]
AU_life_stage_priorities.loc[AU_life_stage_priorities[Life_Stage_Priority_vars[variable_name][life_stage_val]].isin(Life_Stage_Priority)]

In [ ]:
AU_life_stage_priorities.loc[AU_life_stage_priorities[Life_Stage_Priority_vars[variable_name][life_stage_val]].isin(Life_Stage_Priority)]

In [ ]:
str(life_stages_all.loc[i][0]).replace(" ", "_")

In [ ]:

# ------------------ Habitat Attributes for this Life Stage --------------------------------
Life_Stage_Habitat_Criteria = Species_Life_Stage_Habitat_Attributes.loc[Species_Life_Stage_Habitat_Attributes['Life Stage'].isin(Life_Stage_Habitat_Attributes_vars['variable'][life_stage_val_list])]   

# --------------- List of Reaches and Habitat Attributes based on this Life Stage --------------

# ----------------- list of Habitat Attributes for this specific Life Stage ---------------
habitat_attributes_list = Life_Stage_Habitat_Criteria['Habitat Attribute'].values.tolist()
# --------------- Reaches and Habitat Attributes for this Life Stage
Life_Stage_Habitat_Criteria_2 = Species_data_frame_Life_Stage.loc[Species_data_frame_Life_Stage['Habitat Attribute'].isin(habitat_attributes_list)]



In [ ]:
Life_Stage_Habitat_Criteria_2

In [ ]:
Species_data_frame_Life_Stage['Habitat Attribute'].unique()

In [ ]:
habitat_attributes_list